In [225]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib
from scipy import stats
from ydata_profiling import ProfileReport # EDA用ライブラリ

# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from category_encoders import TargetEncoder
# バリデーション
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
# 評価指標
from sklearn.metrics import f1_score, log_loss
# モデリング
import lightgbm as lgb
import xgboost as xgb
from xgboost.callback import EarlyStopping
from catboost import CatBoostClassifier, Pool
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
# パイプライン
from sklearn.pipeline import Pipeline
# パラメータチューニング
import optuna

%matplotlib inline

# pandasのカラムが100列まで見れるようにする
pd.set_option("display.max_columns", 100)

# pandasのレコード数の表示上限を解除
pd.set_option('display.max_rows', None)

## 関数定義

#### データの前処理

In [226]:
### 前処理をまとめた関数
def df_pre1(df):
    # one-hotエンコード(カテゴリのみ)
    # dummy_cols = ["mucous_membrane"]
    # df = pd.get_dummies(df, columns=dummy_cols)

    # labelエンコーディング(2値のみ)
    label_cols = [
        "age",
        "surgical_lesion",
        "cp_data",
        "mucous_membrane",
        "surgery",
    ]
    for col in label_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

    df["pain"] = df["pain"].replace("slight", "moderate")
    df["peristalsis"] = df["peristalsis"].replace("distend_small", "normal")
    df["rectal_exam_feces"] = df["rectal_exam_feces"].replace("serosanguious", "absent")
    df["nasogastric_reflux"] = df["nasogastric_reflux"].replace("slight", "none")

    # 欠損値補完
    # 最頻値で補完
    df["temp_of_extremities"] = df["temp_of_extremities"].fillna("normal")
    df["peripheral_pulse"] = df["peripheral_pulse"].fillna("normal")
    df["capillary_refill_time"] = df["capillary_refill_time"].fillna("3")
    df["pain"] = df["pain"].fillna("depressed")  # どれで補完するか
    df["peristalsis"] = df["peristalsis"].fillna("normal")
    df["abdominal_distention"] = df["abdominal_distention"].fillna("none")
    df["nasogastric_tube"] = df["nasogastric_tube"].fillna("none")
    df["nasogastric_reflux"] = df["nasogastric_reflux"].fillna("none")
    df["rectal_exam_feces"] = df["rectal_exam_feces"].fillna("normal")
    df["abdomen"] = df["abdomen"].fillna("normal")
    df["abdomo_appearance"] = df["abdomo_appearance"].fillna("serosanguious")
    #  "mucous_membrane"はラベルエンコーディングする場合のみ、欠損値補完
    df["mucous_membrane"] = df["mucous_membrane"].fillna("pale_cyanotic")

    # ラベルエンコーディング(マッピング処理)
    df["temp_of_extremities"] = df["temp_of_extremities"].map(
        {"cold": 0, "cool": 1, "normal": 2, "warm": 3}
    )
    df["peripheral_pulse"] = df["peripheral_pulse"].map(
        {"absent": 0, "reduced": 1, "normal": 2, "increased": 3}
    )
    df["capillary_refill_time"] = df["capillary_refill_time"].map(
        {"less_3_sec": 0, "3": 1, "more_3_sec": 2}
    )
    df["pain"] = df["pain"].map(
        {
            "alert": 0,
            # "slight": 1,
            "mild_pain": 1,
            "moderate": 2,
            "depressed": 3,
            "severe_pain": 4,
            "extreme_pain": 5,
        }
    )
    df["peristalsis"] = df["peristalsis"].map(
        {
            "absent": 0,
            "hypomotile": 1,
            "normal": 2,
            "hypermotile": 3,
            # "distend_small": 4,
        }
    )
    df["abdominal_distention"] = df["abdominal_distention"].map(
        {"none": 0, "slight": 1, "moderate": 2, "severe": 3}
    )
    df["nasogastric_tube"] = df["nasogastric_tube"].map(
        {"none": 0, "slight": 1, "significant": 2}
    )
    df["nasogastric_reflux"] = df["nasogastric_reflux"].map(
        {"none": 0, "less_1_liter": 1, "more_1_liter": 2}  # "slight": 1,
    )
    df["rectal_exam_feces"] = df["rectal_exam_feces"].map(
        {
            "absent": 0,
            "decreased": 1,
            "normal": 2,
            "increased": 3,
            # "serosanguious": 4,
        }
    )
    df["abdomen"] = df["abdomen"].map(
        {"normal": 0, "firm": 1, "distend_small": 2, "distend_large": 3, "other": 4}
    )
    df["abdomo_appearance"] = df["abdomo_appearance"].map(
        {"clear": 0, "cloudy": 1, "serosanguious": 2}
    )

    return df

In [227]:
### 前処理をまとめた関数
def df_pre2(df):
    # one-hotエンコード(カテゴリのみ)
    # dummy_cols = ["mucous_membrane"]
    # df = pd.get_dummies(df, columns=dummy_cols)

    # labelエンコーディング(2値のみ)
    label_cols = [
        "age",
        "surgical_lesion",
        # "cp_data",
        "mucous_membrane",
        "surgery",
    ]
    for col in label_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

    df["pain"] = df["pain"].replace("slight", "moderate")
    # df["peristalsis"] = df["peristalsis"].replace("distend_small", "normal")
    df["rectal_exam_feces"] = df["rectal_exam_feces"].replace("serosanguious", "absent")
    df["nasogastric_reflux"] = df["nasogastric_reflux"].replace("slight", "none")

    # 欠損値補完
    # 最頻値で補完
    df["temp_of_extremities"] = df["temp_of_extremities"].fillna("normal")
    # df["peripheral_pulse"] = df["peripheral_pulse"].fillna("normal")
    df["capillary_refill_time"] = df["capillary_refill_time"].fillna("3")
    df["pain"] = df["pain"].fillna("depressed")  # どれで補完するか
    # df["peristalsis"] = df["peristalsis"].fillna("normal")
    df["abdominal_distention"] = df["abdominal_distention"].fillna("none")
    df["nasogastric_tube"] = df["nasogastric_tube"].fillna("none")
    df["nasogastric_reflux"] = df["nasogastric_reflux"].fillna("none")
    df["rectal_exam_feces"] = df["rectal_exam_feces"].fillna("normal")
    # df["abdomen"] = df["abdomen"].fillna("normal")
    df["abdomo_appearance"] = df["abdomo_appearance"].fillna("serosanguious")
    #  "mucous_membrane"はラベルエンコーディングする場合のみ、欠損値補完
    df["mucous_membrane"] = df["mucous_membrane"].fillna("pale_cyanotic")

    # ラベルエンコーディング(マッピング処理)
    df["temp_of_extremities"] = df["temp_of_extremities"].map(
        {"cold": 0, "cool": 1, "normal": 2, "warm": 3}
    )
    # df["peripheral_pulse"] = df["peripheral_pulse"].map(
    #     {"absent": 0, "reduced": 1, "normal": 2, "increased": 3}
    # )
    df["capillary_refill_time"] = df["capillary_refill_time"].map(
        {"less_3_sec": 0, "3": 1, "more_3_sec": 2}
    )
    df["pain"] = df["pain"].map(
        {
            "alert": 0,
            # "slight": 1,
            "mild_pain": 1,
            "moderate": 2,
            "depressed": 3,
            "severe_pain": 4,
            "extreme_pain": 5,
        }
    )
    # df["peristalsis"] = df["peristalsis"].map(
    #     {
    #         "absent": 0,
    #         "hypomotile": 1,
    #         "normal": 2,
    #         "hypermotile": 3,
    #         # "distend_small": 4,
    #     }
    # )
    df["abdominal_distention"] = df["abdominal_distention"].map(
        {"none": 0, "slight": 1, "moderate": 2, "severe": 3}
    )
    df["nasogastric_tube"] = df["nasogastric_tube"].map(
        {"none": 0, "slight": 1, "significant": 2}
    )
    df["nasogastric_reflux"] = df["nasogastric_reflux"].map(
        {"none": 0, "less_1_liter": 1, "more_1_liter": 2}  # "slight": 1,
    )
    df["rectal_exam_feces"] = df["rectal_exam_feces"].map(
        {
            "absent": 0,
            "decreased": 1,
            "normal": 2,
            "increased": 3,
            # "serosanguious": 4,
        }
    )
    # df["abdomen"] = df["abdomen"].map(
    #     {"normal": 0, "firm": 1, "distend_small": 2, "distend_large": 3, "other": 4}
    # )
    df["abdomo_appearance"] = df["abdomo_appearance"].map(
        {"clear": 0, "cloudy": 1, "serosanguious": 2}
    )

    # 追加した特徴量
    df["ng_reflux_ph_cat"] = df["ng_reflux_ph_cat"].map(
        {"low": 0, "normal": 1, "high": 2}
    )

    return df

#### ベースモデル、カスタム評価指標

In [228]:
### マイクロF1スコアを計算する関数(LightGBM カスタム評価指標)


# 引数に、「正解ラベル」と「予測ラベル」を渡す
def micro_f1_metric_lgb(y_true, y_pred):
    # 確率が一番高いクラスを予測ラベルとして取得
    y_pred_labels = np.argmax(y_pred, axis=1)
    score = f1_score(y_true, y_pred_labels, average="micro")

    # 返り値(LightGBM用)：評価指標名, スコア, 指標の最適化方向(Trueで大きいほどモデルの性能が良い)
    return "micro_f1", score, True

In [229]:
### LightGBMのモデル構築・予測を関数化

# パラメータの設定
# params_lgb = {
#     "boosting_type": "gbdt",
#     "objective": "multiclass",
#     "num_class": 3,
#     "metric": "multi_logloss",  # 評価指標にログ損失も使用
#      # "metric": "multi_error",
#     "learning_rate": 0.1,  # 学習時は固定
#     "n_estimators": 10000,  # イテレーション数を増やす
#     "random_state": 123,
#     "importance_type": "gain",
#     "verbose": -1,
#     "num_leaves": 8,
#     "max_depth": 6,
#     # "min_data_in_leaf": 20,  # リーフにおける最小データ数
#     # "feature_fraction": 0.8,  # 特徴量のランダムサンプリング
#     # "bagging_fraction": 0.8,  # サンプルのランダムサンプリング
#     # "lambda_l1": 0.1,  # L1正則化
#     # "lambda_l2": 0.1,  # L2正則化
# }

# Oputunaによるハイパーパラメータチューニング(ベースモデル)
params_lgb1 = {
    "min_child_samples": 24,
    "max_depth": 5,
    "bagging_fraction": 0.7,
    "feature_fraction": 0.6,
    "min_gain_to_split": 0.015142971393036024,
    "verbose": -1,
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_class": 3,
    "metric": "multi_logloss",
    # "metric": "multi_error",
    "importance_type": "gain",
    "random_state": 42,
    "bagging_freq": 1,
    "n_estimators": 20000,
    "lambda_l1": 0.0,
    "lambda_l2": 1.0,
    "learning_rate": 0.1,
}

# Oputunaによるハイパーパラメータチューニング(スタッキング)
params_lgb2 = {
    "learning_rate": 0.01,
    # "learning_rate": 0.21197606382337986,
    "min_child_samples": 18,
    "max_depth": 4,
    "bagging_fraction": 0.8,
    "feature_fraction": 0.85,
    "min_gain_to_split": 2.4257815076675944e-05,
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_class": 3,
    "metric": "multi_logloss",
    "importance_type": "gain",
    "random_state": 42,
    "bagging_freq": 1,
    "n_estimators": 20000,
    "lambda_l1": 0.0,
    "lambda_l2": 1.0,
    "verbose": -1,
}


# モデルの学習と評価を関数化
def lgb_train_eval(input_x, input_y, input_test, params, n_splits=5):
    num_class = 3
    metrics = []
    imp = pd.DataFrame()
    train_pred = np.zeros((input_x.shape[0], num_class))
    test_pred = np.zeros((input_test.shape[0], num_class, n_splits))

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
    cv = list(skf.split(input_x, input_y))

    for nfold in range(n_splits):
        tr_idx, val_idx = cv[nfold][0], cv[nfold][1]
        x_tr, x_val = input_x.iloc[tr_idx], input_x.iloc[val_idx]
        y_tr, y_val = input_y.iloc[tr_idx], input_y.iloc[val_idx]

        model = lgb.LGBMClassifier(**params)
        model.fit(
            x_tr,
            y_tr,
            eval_set=[(x_tr, y_tr), (x_val, y_val)],
            eval_metric=micro_f1_metric_lgb,
            callbacks=[
                lgb.early_stopping(stopping_rounds=100, verbose=True),
                lgb.log_evaluation(100),
            ],
        )

        # 予測結果として、各クラスの確率が出力される
        y_tr_pred = model.predict_proba(x_tr)
        tr_pred_class = np.argmax(y_tr_pred, axis=1)
        y_val_pred = model.predict_proba(x_val)
        y_test_pred = model.predict_proba(input_test)
        # 動作確認用
        # print("訓練データ予測", pd.DataFrame(y_tr_pred))
        # print(pd.DataFrame(tr_pred_class))

        train_pred[val_idx] = y_val_pred
        test_pred[:, :, nfold] = y_test_pred

        # np.argmax(y_tr_pred, axis=1)で、確率が最大のクラス(=予測ラベル)を取得
        tr_score = f1_score(y_tr, np.argmax(y_tr_pred, axis=1), average="micro")
        val_score = f1_score(y_val, np.argmax(y_val_pred, axis=1), average="micro")
        metrics.append([nfold, tr_score, val_score])

        # 動作確認用
        # print("マイクロF1スコア(訓練)：{:.5f}".format(tr_score))

        _imp = pd.DataFrame(
            {
                "columns": input_x.columns,
                "importance": model.feature_importances_,
                "nfold": nfold,
            }
        )
        imp = pd.concat([imp, _imp])

    metrics = np.array(metrics)

    # ラベルを元に戻すための辞書
    label_map = {0: "died", 1: "euthanized", 2: "lived"}
    train_pred = pd.DataFrame(
        train_pred, columns=[f"クラス_{i}_lgb" for i in range(params["num_class"])]
    )
    train_pred["pred_class"] = np.argmax(train_pred.values, axis=1)
    train_pred["pred_class_submit"] = train_pred["pred_class"].map(label_map)

    test_pred = pd.DataFrame(
        np.mean(test_pred, axis=2),
        columns=[f"クラス_{i}_lgb" for i in range(params["num_class"])],
    )
    test_pred["pred_class"] = np.argmax(test_pred.values, axis=1)
    test_pred["pred_class_submit"] = test_pred["pred_class"].map(label_map)

    imp = imp.groupby("columns")["importance"].agg(["mean", "std"]).reset_index()
    imp.columns = ["columns", "imp_mean", "imp_std"]

    return metrics, imp, train_pred, test_pred

In [230]:
# XGBoostのパラメータ設定
# params_xgb = {
#     "objective": "multi:softprob",
#     "num_class": 3,  # クラス数
#     "learning_rate": 0.1,
#     # "eval_metric": "mlogloss",  # 追加でログロスも評価
#     "eval_metric": "merror",  # 誤分類率を評価指標に設定
#     "max_depth": 4,
#     "n_estimators": 10000,
#     "subsample": 0.8,  # サンプルをランダムに選択
#     "colsample_bytree": 0.8,  # 特徴量をランダムに選択
#     "min_child_weight": 3,  # 葉のノードの最小サンプル数
#     # "gamma": 0.1,
#     "random_state": 123,
#     "enable_categorical": True,
#     "early_stopping_rounds": 100,
#     "importance_type": "total_gain",
#     "verbosity": 0,  # ログを非表示
# }

# Oputunaのパラメータチューニング(ベースモデル)
params_xgb1 = {
    "min_child_weight": 0.13963818467308245,
    "max_depth": 8,
    "subsample": 0.75,
    "colsample_bytree": 0.65,
    "gamma": 3.941963789713794e-06,
    "objective": "multi:softprob",
    "num_class": 3,
    "booster": "gbtree",
    # "eval_metric": "mlogloss",
    "eval_metric": "merror",  # 誤分類率を評価指標に設定
    "random_state": 123,
    "importance_type": "total_gain",
    "early_stopping_rounds": 100,
    "n_estimators": 10000,
    "alpha": 0.0,
    "lambda": 1.0,
    "eta": 0.1,
}

# Oputunaのパラメータチューニング(スタッキング)
params_xgb2 = {
    "min_child_weight": 1.9430289780901042,
    "max_depth": 3,
    "subsample": 0.6,
    "colsample_bytree": 0.95,
    "gamma": 0.0004482924032401852,
    "objective": "multi:softprob",
    "num_class": 3,
    "booster": "gbtree",
    "eval_metric": "merror",
    "random_state": 123,
    "importance_type": "total_gain",
    "early_stopping_rounds": 100,
    "n_estimators": 10000,
    "alpha": 0.0,
    "lambda": 1.0,
    "eta": 0.05,
}


# モデルの学習と評価を関数化
def xgb_train_eval(input_x, input_y, input_test, params, n_splits=5):
    num_class = 3
    metrics = []
    imp = pd.DataFrame()
    train_pred = np.zeros((input_x.shape[0], num_class))
    test_pred = np.zeros((input_test.shape[0], num_class, n_splits))

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
    cv = list(skf.split(input_x, input_y))

    for nfold in range(n_splits):
        tr_idx, val_idx = cv[nfold][0], cv[nfold][1]
        x_tr, x_val = input_x.iloc[tr_idx], input_x.iloc[val_idx]
        y_tr, y_val = input_y.iloc[tr_idx], input_y.iloc[val_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(
            x_tr,
            y_tr,
            eval_set=[(x_tr, y_tr), (x_val, y_val)],
        )

        # 予測結果として、各クラスの確率が出力される
        y_tr_pred = model.predict_proba(x_tr)
        tr_pred_class = np.argmax(y_tr_pred, axis=1)
        y_val_pred = model.predict_proba(x_val)
        y_test_pred = model.predict_proba(input_test)
        # 動作確認用
        # print("訓練データ予測", pd.DataFrame(y_tr_pred))
        # print(pd.DataFrame(tr_pred_class))

        train_pred[val_idx] = y_val_pred
        test_pred[:, :, nfold] = y_test_pred

        # np.argmax(y_tr_pred, axis=1)で、確率が最大のクラス(=予測ラベル)を取得
        tr_score = f1_score(y_tr, np.argmax(y_tr_pred, axis=1), average="micro")
        val_score = f1_score(y_val, np.argmax(y_val_pred, axis=1), average="micro")
        metrics.append([nfold, tr_score, val_score])

        # 動作確認用
        # print("マイクロF1スコア(訓練)：{:.5f}".format(tr_score))

        _imp = pd.DataFrame(
            {
                "columns": input_x.columns,
                "importance": model.feature_importances_,
                "nfold": nfold,
            }
        )
        imp = pd.concat([imp, _imp])

    metrics = np.array(metrics)

    # ラベルを元に戻すための辞書
    label_map = {0: "died", 1: "euthanized", 2: "lived"}
    train_pred = pd.DataFrame(
        train_pred, columns=[f"クラス_{i}_xgb" for i in range(params["num_class"])]
    )
    train_pred["pred_class"] = np.argmax(train_pred.values, axis=1)
    train_pred["pred_class_submit"] = train_pred["pred_class"].map(label_map)

    test_pred = pd.DataFrame(
        np.mean(test_pred, axis=2),
        columns=[f"クラス_{i}_xgb" for i in range(params["num_class"])],
    )
    test_pred["pred_class"] = np.argmax(test_pred.values, axis=1)
    test_pred["pred_class_submit"] = test_pred["pred_class"].map(label_map)

    imp = imp.groupby("columns")["importance"].agg(["mean", "std"]).reset_index()
    imp.columns = ["columns", "imp_mean", "imp_std"]

    return metrics, imp, train_pred, test_pred

In [231]:
# ### マイクロF1スコアを計算する関数(XGBoost カスタム評価指標)
# def micro_f1_metric_xgb(y_pred, dtrain):
#     y_true = dtrain.get_label()
#     y_pred_labels = np.argmax(y_pred, axis=1)
#     score = f1_score(y_true, y_pred_labels, average="micro")

#     return "micro_f1", score

In [232]:
# # XGBoost(カスタム評価指標)のパラメータ設定

# params_xgb = {
#     "objective": "multi:softprob",
#     "num_class": 3,  # クラス数
#     "learning_rate": 0.1,
#     "eval_metric": "mlogloss",  # 追加でログロスも評価
#     "max_depth": 4,
#     "n_estimators": 10000,
#     "subsample": 0.8,  # サンプルをランダムに選択
#     "colsample_bytree": 0.8,  # 特徴量をランダムに選択
#     "min_child_weight": 3,  # 葉のノードの最小サンプル数
#     # "gamma": 0.1,
#     "random_state": 123,
#     "enable_categorical": True,
#     "early_stopping_rounds": 100,
#     "importance_type": "total_gain",
#     "verbosity": 0,  # ログを非表示
# }


# def xgb_train_eval(input_x, input_y, input_test, params, n_splits=5):
#     num_class = 3
#     metrics = []
#     imp = pd.DataFrame()
#     train_pred = np.zeros((input_x.shape[0], num_class))
#     test_pred = np.zeros((input_test.shape[0], num_class, n_splits))

#     skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
#     cv = list(skf.split(input_x, input_y))

#     for nfold in range(n_splits):
#         tr_idx, val_idx = cv[nfold]
#         x_tr, x_val = input_x.iloc[tr_idx], input_x.iloc[val_idx]
#         y_tr, y_val = input_y.iloc[tr_idx], input_y.iloc[val_idx]

#         ### ここでdtrain(XGBoostのデータ形式)を作成

#         # データの準備（DMatrix）
#         dtrain = xgb.DMatrix(x_tr, label=y_tr)
#         dval = xgb.DMatrix(x_val, label=y_val)
#         dtest = xgb.DMatrix(input_test)

#         # 訓練
#         model = xgb.train(
#             params,
#             dtrain,
#             num_boost_round=100,
#             evals=[(dtrain, "train"), (dval, "eval")],
#             verbose_eval=100,
#             custom_metric=micro_f1_metric_xgb,
#         )

#         # 予測(確率) ※ "objective"が"multi:softprob"
#         y_tr_pred = model.predict(dtrain)
#         y_val_pred = model.predict(dval)
#         y_test_pred = model.predict(dtest)

#         # 予測結果を保存
#         train_pred[val_idx] = y_val_pred
#         test_pred[:, :, nfold] = y_test_pred

#         # マイクロF1スコアを計算
#         tr_score = f1_score(y_tr, np.argmax(y_tr_pred, axis=1), average="micro")
#         val_score = f1_score(y_val, np.argmax(y_val_pred, axis=1), average="micro")
#         metrics.append([nfold, tr_score, val_score])

#     metrics = np.array(metrics)

#     label_map = {0: "died", 1: "euthanized", 2: "lived"}
#     train_pred = pd.DataFrame(
#         train_pred, columns=[f"クラス_{i}_xgb" for i in range(params["num_class"])]
#     )
#     train_pred["pred_class"] = np.argmax(train_pred.values, axis=1)
#     train_pred["pred_class_submit"] = train_pred["pred_class"].map(label_map)

#     test_pred = pd.DataFrame(
#         np.mean(test_pred, axis=2),
#         columns=[f"クラス_{i}_xgb" for i in range(params["num_class"])],
#     )
#     test_pred["pred_class"] = np.argmax(test_pred.values, axis=1)
#     test_pred["pred_class_submit"] = test_pred["pred_class"].map(label_map)

#     return metrics, train_pred, test_pred

In [ ]:
### CatBoost
# params_cat = {
#     "iterations": 10000,  # 学習の繰り返し回数
#     "learning_rate": 0.1,  # 学習率
#     "depth": 6,  # 木の深さ
#     "loss_function": "MultiClass",  # 多クラス分類
#     "eval_metric": "MultiClass",  # 評価指標
#     # "eval_metric": "TotalF1:average=Micro",
#     # "subsample": 0.8,  # データサンプルの割合
#     # "colsample_bylevel": 0.8,  # 特徴量のサンプル割合
#     # "min_data_in_leaf": 3,  # 最小葉のデータ数
#     "early_stopping_rounds": 100,  # アーリーストッピング
#     "random_state": 123,
#     "verbose": 0,
# }

# optunaによるハイパーパラメータ(privateが高い)
params_cat1 = {
    "learning_rate": 0.1,
    "depth": 8,
    "bagging_temperature": 0.28613933495037946,
    "border_count": 82,
    "min_data_in_leaf": 28,
    "colsample_bylevel": 0.7115532300622305,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    # "colsample_bylevel": 0.8,  # 特徴量のサンプル割合
    # "min_data_in_leaf": 3,  # 最小葉のデータ数
    "iterations": 10000,
    "early_stopping_rounds": 100,
    "random_state": 123,
    "verbose": 0,
}

# Optunaによるハイパーパラメータ(publicが高い)
params_cat2 = {
    "learning_rate": 0.10684874790620232,
    "depth": 5,
    "bagging_temperature": 0.2268514535642031,
    "border_count": 155,
    "min_data_in_leaf": 36,
    "colsample_bylevel": 0.7115532300622305,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "iterations": 10000,
    "early_stopping_rounds": 100,
    "random_state": 123,
    "verbose": 0,
}

# Optunaによるハイパーパラメータ(スタッキング時)
params_cat3 = {
    # "learning_rate": 0.1,  # 学習時は固定
    "learning_rate": 0.01,  # 最高精度
    "depth": 5,
    "bagging_temperature": 0.5777252489999927,
    "border_count": 53,
    "min_data_in_leaf": 17,
    "colsample_bylevel": 0.8987068931775577,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "iterations": 10000,
    "early_stopping_rounds": 100,
    "random_state": 123,
    "verbose": 0,
}

# Optunaによるハイパーパラメータ(スタッキング時)
params_cat4 = {
    "learning_rate": 0.10684874790620232,
    # "learning_rate": 0.01,  # 最高精度
    "depth": 5,
    "bagging_temperature": 0.2268514535642031,
    "border_count": 155,
    "min_data_in_leaf": 36,
    "colsample_bylevel": 0.7115532300622305,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "iterations": 10000,
    "early_stopping_rounds": 100,
    "random_state": 123,
    "verbose": 0,
}

# Optunaによるハイパーパラメータ(カテゴリ変数をそのまま利用)
params_cat5 = {
    "learning_rate": 0.10835456427924503,
    # "learning_rate": 0.01,
    "depth": 3,
    "bagging_temperature": 0.23836663646047773,
    "border_count": 175,
    "min_data_in_leaf": 50,
    "colsample_bylevel": 0.6657674216871736,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "iterations": 10000,
    "early_stopping_rounds": 100,
    "random_state": 123,
    "verbose": 0,
}


def cat_train_eval(
    input_x, input_y, input_test, categorical_features, params, n_splits=5
):
    num_class = 3
    metrics = []
    imp = pd.DataFrame()
    train_pred = np.zeros((input_x.shape[0], num_class))
    test_pred = np.zeros((input_test.shape[0], num_class, n_splits))

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
    cv = list(skf.split(input_x, input_y))

    for nfold in range(n_splits):
        tr_idx, val_idx = cv[nfold]
        x_tr, x_val = input_x.iloc[tr_idx], input_x.iloc[val_idx]
        y_tr, y_val = input_y.iloc[tr_idx], input_y.iloc[val_idx]

        # Poolオブジェクトの作成
        train_pool = Pool(x_tr, y_tr, cat_features=categorical_features)
        val_pool = Pool(x_val, y_val, cat_features=categorical_features)
        test_pool = Pool(input_test, cat_features=categorical_features)

        # モデルの学習
        model = CatBoostClassifier(**params, cat_features=categorical_features)
        model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=100, verbose=100)

        # 予測
        y_tr_pred = model.predict_proba(train_pool)
        y_val_pred = model.predict_proba(val_pool)
        y_test_pred = model.predict_proba(test_pool)

        # 予測結果を保存
        train_pred[val_idx] = y_val_pred
        test_pred[:, :, nfold] = y_test_pred

        # マイクロF1スコアを計算
        tr_score = f1_score(y_tr, np.argmax(y_tr_pred, axis=1), average="micro")
        val_score = f1_score(y_val, np.argmax(y_val_pred, axis=1), average="micro")
        metrics.append([nfold, tr_score, val_score])

        # 特徴量重要度を取得
        _imp = pd.DataFrame(
            {
                "columns": input_x.columns,
                "importance": model.get_feature_importance(type="FeatureImportance"),
                "nfold": nfold,
            }
        )
        imp = pd.concat([imp, _imp])

    metrics = np.array(metrics)

    label_map = {0: "died", 1: "euthanized", 2: "lived"}
    train_pred = pd.DataFrame(
        train_pred, columns=[f"クラス_{i}_cat" for i in range(num_class)]
    )
    train_pred["pred_class"] = np.argmax(train_pred.values, axis=1)
    train_pred["pred_class_submit"] = train_pred["pred_class"].map(label_map)

    test_pred = pd.DataFrame(
        np.mean(test_pred, axis=2),
        columns=[f"クラス_{i}_cat" for i in range(num_class)],
    )
    test_pred["pred_class"] = np.argmax(test_pred.values, axis=1)
    test_pred["pred_class_submit"] = test_pred["pred_class"].map(label_map)

    imp = imp.groupby("columns")["importance"].agg(["mean", "std"]).reset_index()
    imp.columns = ["columns", "imp_mean", "imp_std"]

    return metrics, imp, train_pred, test_pred

In [234]:
### ランダムフォレスト

params_rf = {
    # "n_estimators": 100,  # デフォルト値にして、実行速度を上げる
    "n_estimators": 500,  # 木の数を増やして安定性向上
    "max_depth": None,
    "min_samples_split": 5,  # 分割のための最小サンプル数
    "min_samples_leaf": 2,
    # "max_samples": 0.8,
    "max_features": "sqrt",  # 特徴量はsqrtが分類タスクで効果的
    "min_impurity_decrease": 0.001,
    "class_weight": "balanced",  # クラス不均衡に対応
    "bootstrap": True,  # バギング
    "random_state": 42,
    "n_jobs": -1,  # 並列処理
    "oob_score": True,  # 汎化性能を確認
}


# モデルの学習と評価を関数化
def rf_train_eval(input_x, input_y, input_test, params, n_splits=5):
    num_class = 3  # クラス数を直接指定
    metrics = []
    imp = pd.DataFrame()
    train_pred = np.zeros((input_x.shape[0], num_class))
    test_pred = np.zeros((input_test.shape[0], num_class, n_splits))

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
    cv = list(skf.split(input_x, input_y))

    for nfold in range(n_splits):
        tr_idx, val_idx = cv[nfold][0], cv[nfold][1]
        x_tr, x_val = input_x.iloc[tr_idx], input_x.iloc[val_idx]
        y_tr, y_val = input_y.iloc[tr_idx], input_y.iloc[val_idx]

        model = RandomForestClassifier(**params)
        model.fit(x_tr, y_tr)

        # 予測結果として、各クラスの確率が出力される
        y_tr_pred = model.predict_proba(x_tr)
        tr_pred_class = np.argmax(y_tr_pred, axis=1)
        y_val_pred = model.predict_proba(x_val)
        y_test_pred = model.predict_proba(input_test)
        # 動作確認用
        # print("訓練データ予測", pd.DataFrame(y_tr_pred))
        # print(pd.DataFrame(tr_pred_class))

        train_pred[val_idx] = y_val_pred
        test_pred[:, :, nfold] = y_test_pred

        # np.argmax(y_tr_pred, axis=1)で、確率が最大のクラス(=予測ラベル)を取得
        tr_score = f1_score(y_tr, np.argmax(y_tr_pred, axis=1), average="micro")
        val_score = f1_score(y_val, np.argmax(y_val_pred, axis=1), average="micro")
        metrics.append([nfold, tr_score, val_score])

        # 動作確認用
        # print("マイクロF1スコア(訓練)：{:.5f}".format(tr_score))

        _imp = pd.DataFrame(
            {
                "columns": input_x.columns,
                "importance": model.feature_importances_,
                "nfold": nfold,
            }
        )
        imp = pd.concat([imp, _imp])

    metrics = np.array(metrics)

    # ラベルを元に戻すための辞書
    label_map = {0: "died", 1: "euthanized", 2: "lived"}
    train_pred = pd.DataFrame(
        train_pred, columns=[f"クラス_{i}_rf" for i in range(num_class)]
    )
    train_pred["pred_class"] = np.argmax(train_pred.values, axis=1)
    train_pred["pred_class_submit"] = train_pred["pred_class"].map(label_map)

    test_pred = pd.DataFrame(
        np.mean(test_pred, axis=2),
        columns=[f"クラス_{i}_rf" for i in range(num_class)],
    )
    test_pred["pred_class"] = np.argmax(test_pred.values, axis=1)
    test_pred["pred_class_submit"] = test_pred["pred_class"].map(label_map)

    imp = imp.groupby("columns")["importance"].agg(["mean", "std"]).reset_index()
    imp.columns = ["columns", "imp_mean", "imp_std"]

    return metrics, imp, train_pred, test_pred

In [235]:
### ロジスティック回帰


# モデルの学習と評価を関数化
def lr_train_eval(input_x, input_y, input_test, num_class, n_splits=5):
    metrics = []
    train_pred = np.zeros((input_x.shape[0], num_class))
    test_pred = np.zeros((input_test.shape[0], num_class, n_splits))

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
    cv = list(skf.split(input_x, input_y))

    for nfold in range(n_splits):
        tr_idx, val_idx = cv[nfold][0], cv[nfold][1]
        x_tr, x_val = input_x.iloc[tr_idx], input_x.iloc[val_idx]
        y_tr, y_val = input_y.iloc[tr_idx], input_y.iloc[val_idx]

        # 特徴量の標準化
        scaler = StandardScaler()
        # x_tr = scaler.fit_transform(x_tr)
        # x_val = scaler.transform(x_val)
        # input_test = scaler.transform(input_test)

        x_tr = pd.DataFrame(scaler.fit_transform(x_tr), columns=x_tr.columns)
        x_val = pd.DataFrame(scaler.transform(x_val), columns=x_tr.columns)
        input_test = pd.DataFrame(
            scaler.transform(input_test),
            columns=x_tr.columns,
        )

        # モデルの学習
        model = LogisticRegression(
            C=0.1,
            multi_class="multinomial",
            solver="lbfgs",
            max_iter=1000,
            random_state=123,
        )
        model.fit(x_tr, y_tr)

        # 予測結果として、各クラスの確率が出力される
        y_tr_pred = model.predict_proba(x_tr)
        tr_pred_class = np.argmax(y_tr_pred, axis=1)
        y_val_pred = model.predict_proba(x_val)
        y_test_pred = model.predict_proba(input_test)
        # 動作確認用
        # print("訓練データ予測", pd.DataFrame(y_tr_pred))
        # print(pd.DataFrame(tr_pred_class))

        train_pred[val_idx] = y_val_pred
        test_pred[:, :, nfold] = y_test_pred

        # np.argmax(y_tr_pred, axis=1)で、確率が最大のクラス(=予測ラベル)を取得
        tr_score = f1_score(y_tr, np.argmax(y_tr_pred, axis=1), average="micro")
        val_score = f1_score(y_val, np.argmax(y_val_pred, axis=1), average="micro")
        metrics.append([nfold, tr_score, val_score])

        # 動作確認用
        # print("マイクロF1スコア(訓練)：{:.5f}".format(tr_score))

    metrics = np.array(metrics)

    # ラベルを元に戻すための辞書
    label_map = {0: "died", 1: "euthanized", 2: "lived"}
    train_pred = pd.DataFrame(
        train_pred, columns=[f"クラス_{i}_lr" for i in range(num_class)]
    )
    train_pred["pred_class"] = np.argmax(train_pred.values, axis=1)
    train_pred["pred_class_submit"] = train_pred["pred_class"].map(label_map)

    test_pred = pd.DataFrame(
        np.mean(test_pred, axis=2),
        columns=[f"クラス_{i}_lr" for i in range(num_class)],
    )
    test_pred["pred_class"] = np.argmax(test_pred.values, axis=1)
    test_pred["pred_class_submit"] = test_pred["pred_class"].map(label_map)

    return metrics, train_pred, test_pred

In [236]:
### SVC(RBFカーネル)
# モデルの学習と評価を関数化
def svc_train_eval(input_x, input_y, input_test, num_class, n_splits=5):
    metrics = []
    train_pred = np.zeros((input_x.shape[0], num_class))
    test_pred = np.zeros((input_test.shape[0], num_class, n_splits))

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
    cv = list(skf.split(input_x, input_y))

    for nfold in range(n_splits):
        tr_idx, val_idx = cv[nfold][0], cv[nfold][1]
        x_tr, x_val = input_x.iloc[tr_idx], input_x.iloc[val_idx]
        y_tr, y_val = input_y.iloc[tr_idx], input_y.iloc[val_idx]

        # 特徴量の標準化
        scaler = StandardScaler()
        # x_tr = scaler.fit_transform(x_tr)
        # x_val = scaler.transform(x_val)
        # input_test = scaler.transform(input_test)

        x_tr = pd.DataFrame(scaler.fit_transform(x_tr), columns=x_tr.columns)
        x_val = pd.DataFrame(scaler.transform(x_val), columns=x_tr.columns)
        input_test = pd.DataFrame(
            scaler.transform(input_test),
            columns=x_tr.columns,
        )

        # モデルの学習
        model = SVC(kernel="rbf", C=1.0, gamma="scale", probability=True)
        model.fit(x_tr, y_tr)

        # 予測結果として、各クラスの確率が出力される
        y_tr_pred = model.predict_proba(x_tr)
        tr_pred_class = np.argmax(y_tr_pred, axis=1)
        y_val_pred = model.predict_proba(x_val)
        y_test_pred = model.predict_proba(input_test)
        # 動作確認用
        # print("訓練データ予測", pd.DataFrame(y_tr_pred))
        # print(pd.DataFrame(tr_pred_class))

        train_pred[val_idx] = y_val_pred
        test_pred[:, :, nfold] = y_test_pred

        # np.argmax(y_tr_pred, axis=1)で、確率が最大のクラス(=予測ラベル)を取得
        tr_score = f1_score(y_tr, np.argmax(y_tr_pred, axis=1), average="micro")
        val_score = f1_score(y_val, np.argmax(y_val_pred, axis=1), average="micro")
        metrics.append([nfold, tr_score, val_score])

        # 動作確認用
        # print("マイクロF1スコア(訓練)：{:.5f}".format(tr_score))

    metrics = np.array(metrics)

    # ラベルを元に戻すための辞書
    label_map = {0: "died", 1: "euthanized", 2: "lived"}
    train_pred = pd.DataFrame(
        train_pred, columns=[f"クラス_{i}_lr" for i in range(num_class)]
    )
    train_pred["pred_class"] = np.argmax(train_pred.values, axis=1)
    train_pred["pred_class_submit"] = train_pred["pred_class"].map(label_map)

    test_pred = pd.DataFrame(
        np.mean(test_pred, axis=2),
        columns=[f"クラス_{i}_lr" for i in range(num_class)],
    )
    test_pred["pred_class"] = np.argmax(test_pred.values, axis=1)
    test_pred["pred_class_submit"] = test_pred["pred_class"].map(label_map)

    return metrics, train_pred, test_pred

#### Oputuna

In [237]:
### LightGBM

# 探索しないハイパーパラメータ
params_base_lgb = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_class": 3,
    "metric": "multi_logloss",
    # "metric": "multi_error",
    "importance_type": "gain",
    "random_state": 42,
    "bagging_freq": 1,  # subsampleを有効にするために必要
    "n_estimators": 20000,  # 十分大きくし、アーリーストッピングで制御する
    "lambda_l1": 0.0,
    "lambda_l2": 1.0,
    # 'min_child_samples': 20,
    # 'max_depth': 5,
    # 'bagging_fraction': 0.8,
    # 'feature_fraction': 0.8,
    # 'min_gain_to_split': 0.0,
    "learning_rate": 0.1,  # チューニング時は固定、提出時により小さくする
    "verbose": -1,  # ログ出力なし
}


def objective_lgb(trial):
    # 探索するハイパーパラメータ

    params_tuning = {
        # "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.6, 0.95, step=0.05
        ),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.6, 0.95, step=0.05
        ),
        "min_gain_to_split": trial.suggest_loguniform("min_gain_to_split", 1e-8, 1.0),
        # 余裕があればlambda_l1, lambda_l2も調整する
        # "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 1.0),
        # "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-6, 10.0),
    }
    params_tuning.update(params_base_lgb)

    # モデル学習、評価
    metrics = []  # 評価指標を格納するためのリスト
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
    cv = list(skf.split(x_train, y_train))

    # 交差検証法
    for nfold in range(n_splits):
        tr_idx, val_idx = cv[nfold][0], cv[nfold][1]
        x_tr, x_val = x_train.iloc[tr_idx], x_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        # モデルの学習
        model = lgb.LGBMClassifier(**params_tuning)

        model.fit(
            x_tr,
            y_tr,
            eval_set=[(x_tr, y_tr), (x_val, y_val)],
            eval_metric=micro_f1_metric_lgb,
            callbacks=[
                lgb.early_stopping(stopping_rounds=100, verbose=True),
                lgb.log_evaluation(100),
            ],
        )

        # モデルの予測
        y_val_pred = model.predict(x_val)

        # micro-F1スコアを計算
        val_score = f1_score(y_val, y_val_pred, average="micro")
        metrics.append(val_score)

    return np.mean(metrics)  # 平均スコアを返す

In [238]:
### XGBoost

# ベースラインのパラメータ
params_base_xgb = {
    "objective": "multi:softprob",
    "num_class": 3,  # クラス数
    "booster": "gbtree",
    "eval_metric": "merror",  # 追加でログロスも評価
    "random_state": 123,
    "importance_type": "total_gain",
    "early_stopping_rounds": 100,
    "n_estimators": 10000,  # 十分大きくし、アーリーストッピングで制御する
    # "eta": 0.1,  # チューニング時は固定、提出時により小さくする
    "alpha": 0.0,
    "lambda": 1.0,
    # 'gamma': 0.0,
    # 'min_child_weight': 1,
    # 'max_depth': 5,
    # 'subsample': 0.8,
    # 'colsample_bytree': 0.8,
}


def objective_xgb(trial):
    # 探索するハイパーパラメータ
    # パラメータの探索範囲
    params_tuning = {
        "eta": trial.suggest_loguniform("eta", 0.01, 0.3),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.1, 10),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "subsample": trial.suggest_float("subsample", 0.6, 0.95, step=0.05),
        "colsample_bytree": trial.suggest_float(
            "colsample_bytree", 0.6, 0.95, step=0.05
        ),
        "gamma": trial.suggest_loguniform("gamma", 1e-8, 1.0),
        # 余裕があればalpha, lambdaも調整する
        # 'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        # 'lambda': trial.suggest_loguniform('lambda', 1e-6, 10.0),
    }

    params_tuning.update(params_base_xgb)

    # モデル学習、評価
    metrics = []  # 評価指標を格納するためのリスト
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
    cv = list(skf.split(x_train, y_train))

    # 交差検証法
    for nfold in range(n_splits):
        tr_idx, val_idx = cv[nfold][0], cv[nfold][1]
        x_tr, x_val = x_train.iloc[tr_idx], x_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        model = xgb.XGBClassifier(**params_tuning)
        model.fit(
            x_tr,
            y_tr,
            eval_set=[(x_tr, y_tr), (x_val, y_val)],
        )

        # モデルの予測
        y_val_pred = model.predict(x_val)

        # micro-F1スコアを計算
        val_score = f1_score(y_val, y_val_pred, average="micro")
        metrics.append(val_score)

        ### ここでdtrain(XGBoostのデータ形式)を作成

        # データの準備（DMatrix）
        # dtrain = xgb.DMatrix(x_tr, label=y_tr)
        # dval = xgb.DMatrix(x_val, label=y_val)
        # dtest = xgb.DMatrix(x_test)

        # # 訓練
        # model = xgb.train(
        #     params_tuning,
        #     dtrain,
        #     num_boost_round=100,
        #     evals=[(dtrain, "train"), (dval, "eval")],
        #     verbose_eval=100,
        #     custom_metric=micro_f1_metric_xgb,
        # )

        # 予測(確率) ※ "objective"が"multi:softprob"
        # y_val_pred = model.predict(dval)

        # # micro-F1スコアを計算
        # val_score = f1_score(y_val, np.argmax(y_val_pred, axis=1), average="micro")
        # metrics.append(val_score)

    return np.mean(metrics)  # 平均スコアを返す

In [239]:
### CatBoost
params_base_cat = {
    # "learning_rate": 0.1,  # 学習時は固定
    "loss_function": "MultiClass",  # 多クラス分類
    "eval_metric": "MultiClass",  # 評価指標
    # "eval_metric": "TotalF1:average=Micro",
    "iterations": 10000,  # 学習の繰り返し回数
    "early_stopping_rounds": 100,  # アーリーストッピング
    "random_state": 123,
    "verbose": 0,
    # "l2_leaf_reg": ,
    # "random_strength":
}


def objective_cat(trial):
    # 探索するハイパーパラメータ
    params_tuning = {
        "learning_rate": trial.suggest_float(
            "learning_rate", 0.01, 0.3, log=True
        ),  # 学習率
        "depth": trial.suggest_int("depth", 3, 10),  # 木の深さ
        "bagging_temperature": trial.suggest_float(
            "bagging_temperature", 0.0, 1.0
        ),  # 計算の多様性を高める
        "border_count": trial.suggest_int(
            "border_count", 32, 255
        ),  # OneHotエンコーディングの閾値数
        "min_data_in_leaf": trial.suggest_int(
            "min_data_in_leaf", 1, 50
        ),  # 葉ノードに必要な最小データ数
        "colsample_bylevel": trial.suggest_float(
            "colsample_bylevel", 0.5, 1.0
        ),  # 特徴量のサンプル割合
        # "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-6, 10.0),  # L2正則化
        # "random_strength": trial.suggest_float("random_strength", 0.0, 1.0),  # ランダム性の強さ
    }
    params_tuning.update(params_base_cat)

    # モデル学習、評価
    metrics = []  # 評価指標を格納するためのリスト
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)
    cv = list(skf.split(x_train, y_train))

    # 交差検証法
    for nfold in range(n_splits):
        tr_idx, val_idx = cv[nfold][0], cv[nfold][1]
        x_tr, x_val = x_train.iloc[tr_idx], x_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

        # Poolオブジェクトの作成
        train_pool = Pool(x_tr, y_tr, cat_features=category_cols_cat)
        val_pool = Pool(x_val, y_val, cat_features=category_cols_cat)
        test_pool = Pool(x_test, cat_features=category_cols_cat)

        # モデルの学習
        model = CatBoostClassifier(**params_tuning, cat_features=category_cols_cat)
        model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=100, verbose=100)

        # モデルの予測
        y_val_pred = model.predict(x_val)

        # micro-F1スコアを計算
        val_score = f1_score(y_val, y_val_pred, average="micro")
        metrics.append(val_score)

    return np.mean(metrics)  # 平均スコアを返す

In [240]:
### Oputunaを使い、アンサンブルのWeightを最適化する
def objective_ensemble_weight(trial):
    # # 重みを探索 (0~1 の範囲)
    # w1 = trial.suggest_float("w1", 0, 1)
    # w2 = trial.suggest_float("w2", 0, 1)
    # w3 = trial.suggest_float("w3", 0, 1)

    # # 重みの合計を1に正規化
    # eps = 1e-8  # 小さい値を設定
    # total = max(w1 + w2 + w3, eps)  # 0 になるのを防ぐ
    # w1 = w1 / total
    # w2 = w2 / total
    # w3 = w3 / total

    # # 正規化後の値も記録
    # trial.set_user_attr("w1_norm", w1)
    # trial.set_user_attr("w2_norm", w2)
    # trial.set_user_attr("w3_norm", w3)

    # # 重み付き平均で予測
    # y_pred_ensemble = w1 * model_predict1 + w2 * model_predict2 + w3 * model_predict3

    # # クラスラベルを取得
    # y_pred_labels = np.argmax(y_pred_ensemble, axis=1)

    # # 評価指標 (マイクロF1スコア)
    # score = f1_score(y_true, y_pred_labels, average="micro")
    # return score

    # *******************************************************

    # 重みを探索 (0~1 の範囲)
    w1, w2 = 0, 0
    w1 = trial.suggest_float("w1", 0, 1)
    w2 = trial.suggest_float("w2", 0, 1)

    # 重みの合計を1に正規化
    total2 = w1 + w2
    w1 /= total2
    w2 /= total2

    # 正規化後の値も記録
    trial.set_user_attr("w1_norm", w1)
    trial.set_user_attr("w2_norm", w2)

    # 重み付き平均で予測
    y_pred_ensemble = w1 * model_predict1 + w2 * model_predict2

    # クラスラベルを取得
    y_pred_labels = np.argmax(y_pred_ensemble, axis=1)

    # 評価指標 (マイクロF1スコア)
    score = f1_score(y_true, y_pred_labels, average="micro")

    return score

## データの読み込みと確認

In [241]:
# データセットの読み込み
# 訓練データ
df_train = pd.read_csv("./dataset/train.csv")
train_id = df_train["id"]
del df_train["id"]
print(df_train.shape)
df_train.head(10)

(1235, 28)


,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,NaN,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived
5,no,adult,529642,38.1,56.0,32.0,normal,normal,bright_pink,less_3_sec,depressed,hypomotile,none,none,more_1_liter,5.0,decreased,firm,49.0,8.0,cloudy,2.8,no,0,0,0,yes,lived
6,yes,adult,534787,38.3,36.0,16.0,cool,reduced,normal_pink,less_3_sec,severe_pain,normal,slight,none,more_1_liter,2.0,decreased,NaN,43.0,75.0,cloudy,1.0,no,3111,0,0,yes,euthanized
7,no,adult,529461,39.2,114.0,24.0,cool,reduced,pale_cyanotic,more_3_sec,mild_pain,hypomotile,moderate,slight,none,4.5,absent,distend_small,57.0,7.6,serosanguious,4.5,yes,2207,0,0,yes,died
8,no,adult,528742,37.4,48.0,12.0,cool,reduced,normal_pink,less_3_sec,alert,hypermotile,slight,slight,more_1_liter,7.0,normal,firm,40.0,7.8,cloudy,2.6,no,0,0,0,yes,lived
9,yes,adult,529640,38.3,129.0,48.0,cool,reduced,pale_pink,less_3_sec,depressed,absent,moderate,slight,none,4.4,NaN,NaN,57.0,4.9,cloudy,2.9,yes,3209,0,0,yes,died


In [242]:
df_train["outcome"].value_counts()

outcome
lived         574
died          410
euthanized    251
Name: count, dtype: int64

In [243]:
df_train.isnull().sum()

surgery                    0
age                        0
hospital_number            0
rectal_temp                0
pulse                      0
respiratory_rate           0
temp_of_extremities       39
peripheral_pulse          60
mucous_membrane           21
capillary_refill_time      6
pain                      44
peristalsis               20
abdominal_distention      23
nasogastric_tube          80
nasogastric_reflux        21
nasogastric_reflux_ph      0
rectal_exam_feces        190
abdomen                  213
packed_cell_volume         0
total_protein              0
abdomo_appearance         48
abdomo_protein             0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
outcome                    0
dtype: int64

In [244]:
df_train.dtypes

surgery                   object
age                       object
hospital_number            int64
rectal_temp              float64
pulse                    float64
respiratory_rate         float64
temp_of_extremities       object
peripheral_pulse          object
mucous_membrane           object
capillary_refill_time     object
pain                      object
peristalsis               object
abdominal_distention      object
nasogastric_tube          object
nasogastric_reflux        object
nasogastric_reflux_ph    float64
rectal_exam_feces         object
abdomen                   object
packed_cell_volume       float64
total_protein            float64
abdomo_appearance         object
abdomo_protein           float64
surgical_lesion           object
lesion_1                   int64
lesion_2                   int64
lesion_3                   int64
cp_data                   object
outcome                   object
dtype: object

In [245]:
# # ydata_profiling
# profile = ProfileReport(df_train, title="ydata_profile_horse_health_outcomes")
# profile.to_file("horse_health_outcomes_eda.html")

In [246]:
# テストデータ
df_test = pd.read_csv("./dataset/test.csv")
test_id = df_test["id"]
del df_test["id"]
print(df_test.shape)
df_test.head()

(824, 27)


,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,534053,38.6,40.0,20.0,normal,normal,normal_pink,less_3_sec,mild_pain,hypomotile,slight,none,none,7.0,normal,distend_small,42.0,7.5,clear,2.3,no,0,0,0,no
1,yes,adult,528469,38.2,112.0,48.0,cool,reduced,bright_pink,more_3_sec,depressed,hypomotile,moderate,slight,none,3.5,decreased,distend_small,44.0,6.0,serosanguious,2.6,no,2208,0,0,yes
2,yes,adult,528178,37.7,66.0,12.0,cool,normal,bright_red,less_3_sec,mild_pain,hypomotile,slight,slight,none,3.0,normal,distend_small,31.5,6.0,cloudy,1.6,yes,2205,0,0,yes
3,no,adult,534784,37.1,88.0,20.0,cool,reduced,pale_cyanotic,less_3_sec,depressed,absent,severe,slight,more_1_liter,2.0,absent,distend_large,75.0,81.0,NaN,1.0,yes,1400,0,0,no
4,yes,adult,529840,38.3,50.0,12.0,NaN,normal,bright_pink,less_3_sec,mild_pain,absent,slight,slight,none,3.0,decreased,distend_small,37.0,6.8,cloudy,2.6,yes,2208,0,0,yes


In [247]:
# データの結合
df_concat = pd.concat([df_train, df_test], ignore_index=True)
print(df_concat.shape)
df_concat.head()

(2059, 28)


,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,NaN,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived


In [248]:
df_concat.describe().T

,count,mean,std,min,25%,50%,75%,max
hospital_number,2059.0,1.016073e+06,1.441049e+06,521399.0,528800.0,529796.0,534293.0,5305129.0
rectal_temp,2059.0,3.821914e+01,7.873780e-01,35.4,37.8,38.2,38.6,40.8
pulse,2059.0,7.983633e+01,2.912578e+01,30.0,54.0,76.0,100.0,184.0
respiratory_rate,2059.0,3.032054e+01,1.685000e+01,8.0,18.0,28.0,36.0,96.0
nasogastric_reflux_ph,2059.0,4.432977e+00,1.916504e+00,1.0,2.0,4.5,6.2,7.5
packed_cell_volume,2059.0,4.938669e+01,1.050238e+01,23.0,43.0,48.0,56.0,75.0
total_protein,2059.0,2.115119e+01,2.656672e+01,3.5,6.6,7.5,9.0,89.0
abdomo_protein,2059.0,3.309136e+00,1.569174e+00,0.1,2.0,3.2,4.3,10.1
lesion_1,2059.0,3.783395e+03,5.308602e+03,0.0,2205.0,2209.0,3205.0,41110.0
lesion_2,2059.0,1.372365e+01,1.950135e+02,0.0,0.0,0.0,0.0,4300.0


## データの前処理・特徴量エンジニアリング

In [249]:
# 欠損値があるカラムを抽出
null_cols = df_train.columns[df_train.isnull().sum() > 0]
display(null_cols)

Index(['temp_of_extremities', 'peripheral_pulse', 'mucous_membrane',
       'capillary_refill_time', 'pain', 'peristalsis', 'abdominal_distention',
       'nasogastric_tube', 'nasogastric_reflux', 'rectal_exam_feces',
       'abdomen', 'abdomo_appearance'],
      dtype='object')

In [250]:
### オブジェクト型のカラムを抽出
object_cols = df_train.select_dtypes(include=["object"]).columns
object_cols

common = set(null_cols) & set(object_cols)
print(common)

{'abdominal_distention', 'abdomen', 'capillary_refill_time', 'temp_of_extremities', 'mucous_membrane', 'peripheral_pulse', 'rectal_exam_feces', 'pain', 'peristalsis', 'nasogastric_tube', 'abdomo_appearance', 'nasogastric_reflux'}


In [251]:
df_concat[object_cols].head()

,surgery,age,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,rectal_exam_feces,abdomen,abdomo_appearance,surgical_lesion,cp_data,outcome
0,yes,adult,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,decreased,distend_small,serosanguious,yes,no,died
1,yes,adult,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,absent,distend_small,serosanguious,yes,no,euthanized
2,yes,adult,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,NaN,distend_large,serosanguious,yes,no,lived
3,yes,adult,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,decreased,distend_small,cloudy,yes,yes,lived
4,no,adult,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,normal,normal,cloudy,no,yes,lived


In [252]:
### 目的変数をラベルエンコーディング

# 元々の目的変数を確認
# display(df_train["outcome"].head())

# ラベルエンコーディングのインスタンスを作成
label_encoder = LabelEncoder()

# 目的変数をラベルエンコーディング
df_train["outcome"] = label_encoder.fit_transform(df_train["outcome"])
df_train.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,0
1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,1
2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,NaN,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,2
3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,2
4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,0,0,yes,2


#### EDA・可視化

In [253]:
# # カテゴリごとの出現回数を集計
# class_counts = df_train["outcome"].value_counts()

# label_map = {0: "died", 1: "euthanized", 2: "lived"}
# class_labels = class_counts.index.map(label_map)

# # 円グラフを描画
# plt.figure(figsize=(6, 6))
# plt.pie(
#     class_counts,
#     # labels=["died", "euthanized", "lived"],
#     labels=class_labels,
#     autopct="%1.1f%%",  # 割合を表示
#     startangle=90,  # 開始角度
#     colors=["red", "blue", "yellow"],  # 任意の色
# )

# plt.title("目的変数Outcomeの割合(trainデータ)")
# plt.show()

In [254]:
### 特徴量を数値型とカテゴリ型に分ける

# カテゴリ型のカラム
category_cols = df_train.select_dtypes(include=["object"]).columns
# df_category_cols = df_train[category_cols]
# print(df_category_cols.shape)
# df_category_cols.head()

# 数値型のカラム
num_cols = df_train.select_dtypes(include=["float64", "int64"]).columns
# df_num_cols = df_train[num_cols].drop(columns=["outcome"])
# df_num_cols.head()

#### カテゴリ型変数

In [255]:
print(len(category_cols))

16


In [256]:
# plt.figure(figsize=(14, len(category_cols) * 3))

# for i, col in enumerate(category_cols):
#     # 行数：len(category_cols) // 2 + len(category_cols) % 2
#     # len(category_cols) // 2の部分で 1行に2つずつ
#     # len(category_cols) % 2の部分で、行数が奇数の場合に1行追加
#     # 列数：2
#     # i + 1で、1から始まるようにする
#     plt.subplot(len(category_cols) // 2 + len(category_cols) % 2, 2, i + 1)
#     sns.countplot(x=col, hue="outcome", data=df_train, palette="muted")
#     plt.ylim(0, df_train[col].value_counts().max() + 10)

# plt.tight_layout()
# plt.show()

# ### one-hot(ダミー変数)
# # mucous_membrane

# ### label(順序) → 2値のみ
# # surgery, age, surgical_lesion, cp_data

# ### label(順序) → 3値以上

#### 数値型変数

In [257]:
# plt.figure(figsize=(14, len(num_cols) * 3))

# for i, col in enumerate(num_cols):
#     plt.subplot(len(num_cols), 2, i + 1)

#     sns.histplot(
#         x=col, hue="outcome", data=df_train, kde=True, bins=50, palette="muted"
#     )
#     plt.title("Histplot by outcome(目的変数)")
#     plt.ylim(0, df_train[col].value_counts().max() + 10)

# plt.tight_layout()
# plt.show()

In [258]:
# ### 目的変数との相関係数を計算
# target = "outcome"
# correlatin = df_train[num_cols].corr()[target]
# correlation_matrix = df_train[num_cols].corr()

# # ヒートマップを作成
# plt.figure(figsize=(10, 8))
# sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1, center=0)
# plt.title("数値型カラム間の相関係数")
# plt.show()

#### 外れ値

In [259]:
# # サブプロットの作成
# fig, axes = plt.subplots(1, len(num_cols), figsize=(20, 10))

# # 各数値型カラムに対して箱ひげ図を描画
# for i, column in enumerate(num_cols):
#     if column not in ["outcome", "hospital_number", "lesion_2", "lesion_3"]:
#         df_concat.boxplot(column=column, ax=axes[i])
#         axes[i].set_title(f"Box Plot of {column}")
#         axes[i].set_ylabel("Values")

In [260]:
### 外れ値を検出(Zスコア)
def outiler_z_score(df, column, threshold=3):
    z_score = np.abs(stats.zscore(df[column]))
    outliers = df[z_score > threshold]
    return outliers


# for column in num_cols:
#     if column not in ["outcome", "hospital_number", "lesion_2", "lesion_3"]:
#         outliers = outiler_z_score(df_concat, column, threshold=3)
#         print(f"{column}の外れ値数: {len(outliers)}")
#         # 外れ値があるインデックスと外れ値のみを出力
#         if len(outliers) > 0:
#             print("外れ値のインデックスと値:")
#             for index, value in outliers[column].items():
#                 print(f"インデックス: {index}, 値: {value}")

In [261]:
### 外れ値を検出(IQR法)
# 外れ値を検出する関数
def outiler_iqr(df, column, threshold=1.5):
    # 四分位数を計算
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    # 外れ値の閾値を計算
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr
    # 外れ値を抽出
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound


# for column in num_cols:
#     if column not in ["outcome", "hospital_number", "lesion_2", "lesion_3"]:
#         outliers, lower, upper = outiler_iqr(df_concat, column, 2)
#         print("{}の外れ値の下限：{:.5f}".format(column, lower))
#         print("{}の外れ値の上限：{:.5f}".format(column, upper))
#         print("{}の外れ値：{}個".format(column, len(outliers)))
# 外れ値があるインデックスと外れ値のみを出力
# if len(outliers) > 0:
#     print("外れ値のインデックスと値:")
#     for index, value in outliers[column].items():
#         print(f"インデックス: {index}, 値: {value}")

In [262]:
### 外れ値について個別に見ていく
search_column = "respiratory_rate"

# zスコアによる外れ値の検出
out_z = outiler_z_score(df_concat, search_column, threshold=3)
print(f"{search_column}の外れ値数: {len(out_z)}")
# if len(out_z) > 0:
#     print("外れ値のインデックスと値:")
#     for index, value in out_z[search_column].items():
#         print(f"インデックス: {index}, 値: {value}")

respiratory_rateの外れ値数: 69


In [263]:
# IQR法による外れ値の検出
out_iqr, lower, upper = outiler_iqr(df_concat, search_column, threshold=2)
print("{}の外れ値の下限：{:.5f}".format(search_column, lower))
print("{}の外れ値の上限：{:.5f}".format(search_column, upper))
print("{}の外れ値：{}個".format(search_column, len(out_iqr)))
# 外れ値があるインデックスと外れ値のみを出力
# if len(out_iqr) > 0:
#     print("外れ値のインデックスと値:")
#     for index, value in out_iqr[search_column].items():
#         print(f"インデックス: {index}, 値: {value}")

respiratory_rateの外れ値の下限：-18.00000
respiratory_rateの外れ値の上限：72.00000
respiratory_rateの外れ値：81個


In [264]:
# "respiratory_rate"が90以上のデータを抽出
df_90 = df_concat[df_concat["respiratory_rate"] >= 90]
print(len(df_90))
# display(df_90["respiratory_rate"])

30


## データセットの定義

In [265]:
df_concat.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,NaN,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived


#### CatBoost用

In [266]:
# 新しい特徴量
df_new1 = pd.DataFrame()

# ショック指数
df_new1["shock_index"] = df_concat["pulse"] / df_concat["respiratory_rate"]


# # 高熱かどうか → "rectal_temp"をそのまま使うほうがよい
# df_new1["fever_score"] = df_concat["rectal_temp"] - 37.5

# df_new1["critical_vital_signs_score"] = (df_new1["shock_index"] > 1.5).astype(int) + (
#     df_new1["fever_score"] > 1
# ).astype(int)


#  2. 痛みと直腸温の相互作用
# df['pain_temp_interaction'] = df['pain_encoded'] * df['rectal_temp']

# 3. 腹水タンパクと総タンパクの比率
# df_new1["protein_ratio"] = df_concat["abdomo_protein"] / df_concat["total_protein"]

# 4. ショック指数
# df['shock_index'] = df['pulse'] / df['respiratory_rate']

# 5. 経鼻胃管逆流pHのカテゴリ化
# df_new1["ng_reflux_ph_cat"] = pd.cut(
#     df_concat["nasogastric_reflux_ph"],
#     bins=[0, 3, 7, 14],
#     labels=["low", "normal", "high"],
# )

In [267]:
### 特徴量重要度が低い特徴量を削除
cols_to_drop1 = [
    # 新たに追加した特徴量のうち、もとの特徴量は削除
    # "rectal_temp",
    # "pulse",
    # "respiratory_rate",
    # "cp_data",
    # "peripheral_pulse",
    # "peristalsis",
    # "abdomen",
    # 多重共線性がある特徴量
    "lesion_2",
    # "lesion_3",
    # "surgery",  # surgical_lesionと高い相関がある、surgical_lesionを残した方が良いっぽい
    # "surgical_lesion"
]

In [268]:
# ### 使用するデータセット1：CatBoost
df_concat1 = df_concat.copy()

# # 一部の特徴量を削除
df_concat1 = df_concat1.drop(columns=cols_to_drop1)
del df_concat1["outcome"]
df_concat1 = pd.concat([df_concat1, df_train["outcome"]], axis=1)
# display(df_concat1.head(), df_concat1.shape)

# 追加した特徴量を結合
df_concat1 = pd.concat([df_concat1, df_new1], axis=1)


display(df_concat1.head(), df_concat1.shape, df_concat1.isnull().sum().sum())

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_3,cp_data,outcome,shock_index
0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,no,0.0,5.500000
1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,no,1.0,7.333333
2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,NaN,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,no,2.0,4.285714
3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,yes,2.0,2.400000
4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,0,yes,2.0,1.083333


(2059, 28)

2148

In [269]:
df_encoded1 = df_pre1(df_concat1)
display(df_encoded1.head())

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_3,cp_data,outcome,shock_index
0,1,0,530001,38.1,132.0,24.0,1,1,2,2,3,0,1,1,1,6.5,1,2,57.0,8.5,2,3.4,1,2209,0,0,0.0,5.500000
1,1,0,533836,37.5,88.0,12.0,1,2,4,2,1,0,2,0,2,2.0,0,2,33.0,64.0,2,2.0,1,2208,0,0,1.0,7.333333
2,1,0,529812,38.3,120.0,28.0,1,1,5,0,5,1,2,1,0,3.5,2,3,37.0,6.4,2,3.4,1,5124,0,0,2.0,4.285714
3,1,0,5262541,37.1,72.0,30.0,0,1,5,2,1,1,2,1,2,2.0,1,2,53.0,7.0,1,3.9,1,2208,0,1,2.0,2.400000
4,0,0,5299629,38.0,52.0,48.0,2,2,3,0,0,1,0,1,1,7.0,2,0,47.0,7.3,1,2.6,0,0,0,1,2.0,1.083333


In [270]:
## ターゲットエンコーディング
# 1. train/test の分割
df_train_te1 = df_encoded1[df_encoded1["outcome"].notna()].copy()  # trainデータ
df_test_te1 = df_encoded1[
    df_encoded1["outcome"].isna()
].copy()  # testデータ（予測対象）

# 2. KFold Target Encoding を適用（自己リークを防ぐ）
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 初期化
df_train_te1["hospital_number_encoded"] = np.nan
df_train_te1["lesion_1_encoded"] = np.nan  # 統一
# df_train_te1["hospital_mortality_rate"] = np.nan

for train_idx, valid_idx in kf.split(df_train_te1):
    # 各 fold の学習データ・バリデーションデータを分割
    train_fold = df_train_te1.iloc[train_idx]
    valid_fold = df_train_te1.iloc[valid_idx]

    # hospital_mortality_map = train_fold.groupby("hospital_number")["outcome"].apply(
    #     lambda x: (x == 0).mean()
    # )
    # df_train_te1.loc[valid_idx, "hospital_mortality_rate"] = (
    #     df_train_te1.loc[valid_idx]["hospital_number"]
    #     .map(hospital_mortality_map)
    #     .fillna(0)
    # )

    # ターゲットエンコーダーの学習（目的変数を pd.Series に変換）
    encoder = TargetEncoder(cols=["hospital_number", "lesion_1"])
    encoder.fit(
        train_fold[["hospital_number", "lesion_1"]], pd.Series(train_fold["outcome"])
    )

    # valid_fold に対してエンコーディングを適用（データフレームに変換）
    df_train_te1.loc[valid_idx, ["hospital_number_encoded", "lesion_1_encoded"]] = (
        encoder.transform(valid_fold[["hospital_number", "lesion_1"]]).values
    )

# 3. test データには、train データ全体で学習したエンコーダーを適用
encoder = TargetEncoder(cols=["hospital_number", "lesion_1"])
encoder.fit(
    df_train_te1[["hospital_number", "lesion_1"]], pd.Series(df_train_te1["outcome"])
)

df_test_te1[["hospital_number_encoded", "lesion_1_encoded"]] = encoder.transform(
    df_test_te1[["hospital_number", "lesion_1"]]
).values

# # testデータには train の値を適用
# df_test_te1["hospital_mortality_rate"] = (
#     df_test_te1["hospital_number"]
#     .map(
#         df_train_te1.groupby("hospital_number")["outcome"].apply(
#             lambda x: (x == 0).mean()
#         )
#     )
#     .fillna(0)
# )

# 4. train/test を再結合
df_encoded1 = pd.concat([df_train_te1, df_test_te1], axis=0).sort_index()

# 確認
display(df_encoded1.head())

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_3,cp_data,outcome,shock_index,hospital_number_encoded,lesion_1_encoded
0,1,0,530001,38.1,132.0,24.0,1,1,2,2,3,0,1,1,1,6.5,1,2,57.0,8.5,2,3.4,1,2209,0,0,0.0,5.500000,0.908885,1.074200
1,1,0,533836,37.5,88.0,12.0,1,2,4,2,1,0,2,0,2,2.0,0,2,33.0,64.0,2,2.0,1,2208,0,0,1.0,7.333333,1.256374,1.008410
2,1,0,529812,38.3,120.0,28.0,1,1,5,0,5,1,2,1,0,3.5,2,3,37.0,6.4,2,3.4,1,5124,0,0,2.0,4.285714,1.146761,0.997558
3,1,0,5262541,37.1,72.0,30.0,0,1,5,2,1,1,2,1,2,2.0,1,2,53.0,7.0,1,3.9,1,2208,0,1,2.0,2.400000,1.146761,0.984256
4,0,0,5299629,38.0,52.0,48.0,2,2,3,0,0,1,0,1,1,7.0,2,0,47.0,7.3,1,2.6,0,0,0,1,2.0,1.083333,1.271710,1.860710


In [271]:
### 説明変数と目的変数を定義
df_concat1 = df_encoded1.copy()


# 歪度が大きいカラムを対数変換
# np.log1pを使うことで、log(1+x)を計算する
# df_concat1["respiratory_rate"] = np.log1p(df_concat1["respiratory_rate"])
# df_concat1["total_protein"] = np.log1p(df_concat1["total_protein"])
# df_concat1["abdomo_protein"] = np.log1p(df_concat1["abdomo_protein"])

# "leison_2"のカラムについて、0より大きいなら1、それ以外は0
# df_concat1["lesion_2"] = df_concat1["lesion_2"].apply(lambda x: 1 if x > 0 else 0)
# df_concat1["lesion_2"].value_counts()

# そのままhospital_numberを使うか、hospital_number_encodedを使うかは後で決める
# del df_concat1["hospital_number"]
# del df_concat1["hospital_number_encoded"]

# # そのままlesion_1を使うか、lesion_1_encodedを使うかは後で決める
# del df_concat1["lesion_1"]
# del df_concat1["lesion_1_encoded"]

# del df_concat1["total_protein"]
# del df_concat1["abdomo_protein"]


# 訓練データとテストデータに再度分割
df_train1, df_test1 = df_concat1[:1235], df_concat1[1235:]

# 説明変数
if "outcome" in df_train1.columns:
    x_train1 = df_train1.drop(columns=["outcome"])
else:
    x_train1 = df_train1

y_train1 = df_train1["outcome"]
print("訓練データ", x_train1.shape, y_train1.shape)
# print(y_train1.isnull().sum())

# 目的変数
if "outcome" in df_test1.columns:
    x_test1 = df_test1.drop(columns=["outcome"])
else:
    x_test1 = df_test1
print("テストデータ", x_test1.shape)

訓練データ (1235, 29) (1235,)
テストデータ (824, 29)


In [272]:
display(x_train1.head())
# x_train1.isnull().sum()

# カテゴリ変数を取得
category_cols_cat = x_train1.select_dtypes(include=["object"]).columns.to_list()
print(category_cols_cat)
print(len(category_cols_cat))
# x_train1["ng_reflux_ph_cat"] = x_train1["ng_reflux_ph_cat"].astype("int")
# x_test1["ng_reflux_ph_cat"] = x_test1["ng_reflux_ph_cat"].astype("int")

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_3,cp_data,shock_index,hospital_number_encoded,lesion_1_encoded
0,1,0,530001,38.1,132.0,24.0,1,1,2,2,3,0,1,1,1,6.5,1,2,57.0,8.5,2,3.4,1,2209,0,0,5.500000,0.908885,1.074200
1,1,0,533836,37.5,88.0,12.0,1,2,4,2,1,0,2,0,2,2.0,0,2,33.0,64.0,2,2.0,1,2208,0,0,7.333333,1.256374,1.008410
2,1,0,529812,38.3,120.0,28.0,1,1,5,0,5,1,2,1,0,3.5,2,3,37.0,6.4,2,3.4,1,5124,0,0,4.285714,1.146761,0.997558
3,1,0,5262541,37.1,72.0,30.0,0,1,5,2,1,1,2,1,2,2.0,1,2,53.0,7.0,1,3.9,1,2208,0,1,2.400000,1.146761,0.984256
4,0,0,5299629,38.0,52.0,48.0,2,2,3,0,0,1,0,1,1,7.0,2,0,47.0,7.3,1,2.6,0,0,0,1,1.083333,1.271710,1.860710


[]
0


#### CatBoost(カテゴリ変数そのまま)

In [273]:
# ### 使用するデータセット1：CatBoost
df_concat3 = df_concat.copy()

# # 一部の特徴量を削除
df_concat3 = df_concat3.drop(columns=cols_to_drop1)
del df_concat3["outcome"]
df_concat3 = pd.concat([df_concat3, df_train["outcome"]], axis=1)
# display(df_concat3.head(), df_concat3.shape)

# 追加した特徴量を結合
df_concat3 = pd.concat([df_concat3, df_new1], axis=1)
df_concat3 = pd.concat(
    [df_concat3, df_concat1[["hospital_number_encoded", "lesion_1_encoded"]]], axis=1
)

In [274]:
df_concat3["pain"] = df_concat3["pain"].replace("slight", "moderate")
df_concat3["peristalsis"] = df_concat3["peristalsis"].replace("distend_small", "normal")
df_concat3["rectal_exam_feces"] = df_concat3["rectal_exam_feces"].replace(
    "serosanguious", "absent"
)
df_concat3["nasogastric_reflux"] = df_concat3["nasogastric_reflux"].replace(
    "slight", "none"
)

# 欠損値補完
# 最頻値で補完
df_concat3["temp_of_extremities"] = df_concat3["temp_of_extremities"].fillna("normal")
df_concat3["peripheral_pulse"] = df_concat3["peripheral_pulse"].fillna("normal")
df_concat3["capillary_refill_time"] = df_concat3["capillary_refill_time"].fillna("3")
df_concat3["pain"] = df_concat3["pain"].fillna("depressed")  # どれで補完するか
df_concat3["peristalsis"] = df_concat3["peristalsis"].fillna("normal")
df_concat3["abdominal_distention"] = df_concat3["abdominal_distention"].fillna("none")
df_concat3["nasogastric_tube"] = df_concat3["nasogastric_tube"].fillna("none")
df_concat3["nasogastric_reflux"] = df_concat3["nasogastric_reflux"].fillna("none")
df_concat3["rectal_exam_feces"] = df_concat3["rectal_exam_feces"].fillna("normal")
df_concat3["abdomen"] = df_concat3["abdomen"].fillna("normal")
df_concat3["abdomo_appearance"] = df_concat3["abdomo_appearance"].fillna(
    "serosanguious"
)
#  "mucous_membrane"はラベルエンコーディングする場合のみ、欠損値補完
df_concat3["mucous_membrane"] = df_concat3["mucous_membrane"].fillna("pale_cyanotic")

In [275]:
display(df_concat3.head(), df_concat3.shape, df_concat3.isnull().sum().sum())

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_3,cp_data,outcome,shock_index,hospital_number_encoded,lesion_1_encoded
0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,no,0.0,5.500000,0.908885,1.074200
1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,no,1.0,7.333333,1.256374,1.008410
2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,normal,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,no,2.0,4.285714,1.146761,0.997558
3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,yes,2.0,2.400000,1.146761,0.984256
4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,0,yes,2.0,1.083333,1.271710,1.860710


(2059, 30)

824

In [276]:
# 訓練データとテストデータに再度分割
df_train3, df_test3 = df_concat3[:1235], df_concat3[1235:]

# 説明変数
if "outcome" in df_train3.columns:
    x_train3 = df_train3.drop(columns=["outcome"])
else:
    x_train3 = df_train3

y_train3 = df_train3["outcome"]
print("訓練データ", x_train3.shape, y_train3.shape)
# print(y_train3.isnull().sum())

# 目的変数
if "outcome" in df_test3.columns:
    x_test3 = df_test3.drop(columns=["outcome"])
else:
    x_test3 = df_test3
print("テストデータ", x_test3.shape)

# カテゴリ変数を取得
category_cols_cat = x_train3.select_dtypes(include=["object"]).columns.to_list()
print(category_cols_cat)
print(len(category_cols_cat))

訓練データ (1235, 29) (1235,)
テストデータ (824, 29)
['surgery', 'age', 'temp_of_extremities', 'peripheral_pulse', 'mucous_membrane', 'capillary_refill_time', 'pain', 'peristalsis', 'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux', 'rectal_exam_feces', 'abdomen', 'abdomo_appearance', 'surgical_lesion', 'cp_data']
16


In [277]:
x_train3.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_3,cp_data,shock_index,hospital_number_encoded,lesion_1_encoded
0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,no,5.500000,0.908885,1.074200
1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,no,7.333333,1.256374,1.008410
2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,normal,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,no,4.285714,1.146761,0.997558
3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,yes,2.400000,1.146761,0.984256
4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,0,yes,1.083333,1.271710,1.860710


#### LightGBM用

In [278]:
# 新しい特徴量
df_new2 = pd.DataFrame()

# ショック指数
df_new2["shock_index"] = df_concat["pulse"] / df_concat["respiratory_rate"]


# # 高熱かどうか → "rectal_temp"をそのまま使うほうがよい
# df_new2["fever_score"] = df_concat["rectal_temp"] - 37.5

# df_new2["critical_vital_signs_score"] = (df_new2["shock_index"] > 1.5).astype(int) + (
#     df_new2["fever_score"] > 1
# ).astype(int)


#  2. 痛みと直腸温の相互作用
# df['pain_temp_interaction'] = df['pain_encoded'] * df['rectal_temp']

# 3. 腹水タンパクと総タンパクの比率
df_new2["protein_ratio"] = df_concat["abdomo_protein"] / df_concat["total_protein"]

# 4. ショック指数
# df['shock_index'] = df['pulse'] / df['respiratory_rate']

# 5. 経鼻胃管逆流pHのカテゴリ化
df_new2["ng_reflux_ph_cat"] = pd.cut(
    df_concat["nasogastric_reflux_ph"],
    # bins=[0, 4, 7, 14],
    bins=[0, 3, 7, 14],
    labels=["low", "normal", "high"],
    # labels=[0, 1, 2],
)

In [279]:
### 特徴量重要度が低い特徴量を削除
cols_to_drop2 = [
    # 新たに追加した特徴量のうち、もとの特徴量は削除
    # "nasogastric_reflux_ph",
    # "abdomo_protein",
    # "total_protein",
    # "pulse",
    # "respiratory_rate",
    "cp_data",
    "peripheral_pulse",
    "peristalsis",
    "abdomen",
    # 多重共線性がある特徴量
    "lesion_2",
    "lesion_3",
    # "surgery",  # surgical_lesionと高い相関がある、surgical_lesionを残した方が良いっぽい
    # "surgical_lesion"
]

In [280]:
# ### 使用するデータセット2：LightGBMのデータセット
df_concat2 = df_concat.copy()

# # 一部の特徴量を削除
df_concat2 = df_concat2.drop(columns=cols_to_drop2)
del df_concat2["outcome"]
df_concat2 = pd.concat([df_concat2, df_train["outcome"]], axis=1)
# display(df_concat2.head(), df_concat2.shape)

# 追加した特徴量を結合
df_concat2 = pd.concat([df_concat2, df_new2], axis=1)


# display(df_concat2.head(), df_concat2.shape, df_concat2.isnull().sum().sum())

In [281]:
df_encoded2 = df_pre2(df_concat2)
display(df_encoded2.head())

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,mucous_membrane,capillary_refill_time,pain,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,outcome,shock_index,protein_ratio,ng_reflux_ph_cat
0,1,0,530001,38.1,132.0,24.0,1,2,2,3,1,1,1,6.5,1,57.0,8.5,2,3.4,1,2209,0.0,5.500000,0.400000,1
1,1,0,533836,37.5,88.0,12.0,1,4,2,1,2,0,2,2.0,0,33.0,64.0,2,2.0,1,2208,1.0,7.333333,0.031250,0
2,1,0,529812,38.3,120.0,28.0,1,5,0,5,2,1,0,3.5,2,37.0,6.4,2,3.4,1,5124,2.0,4.285714,0.531250,1
3,1,0,5262541,37.1,72.0,30.0,0,5,2,1,2,1,2,2.0,1,53.0,7.0,1,3.9,1,2208,2.0,2.400000,0.557143,0
4,0,0,5299629,38.0,52.0,48.0,2,3,0,0,0,1,1,7.0,2,47.0,7.3,1,2.6,0,0,2.0,1.083333,0.356164,1


In [282]:
## ターゲットエンコーディング
# 1. train/test の分割
df_train_te2 = df_encoded2[df_encoded2["outcome"].notna()].copy()  # trainデータ
df_test_te2 = df_encoded2[
    df_encoded2["outcome"].isna()
].copy()  # testデータ（予測対象）

# 2. KFold Target Encoding を適用（自己リークを防ぐ）
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 初期化
df_train_te2["hospital_number_encoded"] = np.nan
df_train_te2["lesion_1_encoded"] = np.nan  # 統一
df_train_te2["hospital_mortality_rate"] = np.nan

for train_idx, valid_idx in kf.split(df_train_te2):
    # 各 fold の学習データ・バリデーションデータを分割
    train_fold = df_train_te2.iloc[train_idx]
    valid_fold = df_train_te2.iloc[valid_idx]

    hospital_mortality_map = train_fold.groupby("hospital_number")["outcome"].apply(
        lambda x: (x == 0).mean()
    )
    df_train_te2.loc[valid_idx, "hospital_mortality_rate"] = (
        df_train_te2.loc[valid_idx]["hospital_number"]
        .map(hospital_mortality_map)
        .fillna(0)
    )

    # ターゲットエンコーダーの学習（目的変数を pd.Series に変換）
    encoder = TargetEncoder(cols=["hospital_number", "lesion_1"])
    encoder.fit(
        train_fold[["hospital_number", "lesion_1"]], pd.Series(train_fold["outcome"])
    )

    # valid_fold に対してエンコーディングを適用（データフレームに変換）
    df_train_te2.loc[valid_idx, ["hospital_number_encoded", "lesion_1_encoded"]] = (
        encoder.transform(valid_fold[["hospital_number", "lesion_1"]]).values
    )

# 3. test データには、train データ全体で学習したエンコーダーを適用
encoder = TargetEncoder(cols=["hospital_number", "lesion_1"])
encoder.fit(
    df_train_te2[["hospital_number", "lesion_1"]], pd.Series(df_train_te2["outcome"])
)

df_test_te2[["hospital_number_encoded", "lesion_1_encoded"]] = encoder.transform(
    df_test_te2[["hospital_number", "lesion_1"]]
).values

# testデータには train の値を適用
df_test_te2["hospital_mortality_rate"] = (
    df_test_te2["hospital_number"]
    .map(
        df_train_te2.groupby("hospital_number")["outcome"].apply(
            lambda x: (x == 0).mean()
        )
    )
    .fillna(0)
)

# 4. train/test を再結合
df_encoded2 = pd.concat([df_train_te2, df_test_te2], axis=0).sort_index()

# 確認
display(df_encoded2.head())

# # NaN になっている hospital_number を特定
# nan_hospitals = df_train_te2[df_train_te2["hospital_mortality_rate"].isna()]["hospital_number"].unique()
# print("NaN が発生している hospital_number:", nan_hospitals)

# # 各 fold で hospital_number のカバー率を確認
# for train_idx, valid_idx in kf.split(df_train_te2):
#     train_fold = df_train_te2.iloc[train_idx]
#     valid_fold = df_train_te2.iloc[valid_idx]

#     train_hospitals = set(train_fold["hospital_number"].unique())
#     valid_hospitals = set(valid_fold["hospital_number"].unique())

#     missing_hospitals = valid_hospitals - train_hospitals
#     print(f"Fold で学習データにない hospital_number: {missing_hospitals}")

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,mucous_membrane,capillary_refill_time,pain,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,outcome,shock_index,protein_ratio,ng_reflux_ph_cat,hospital_number_encoded,lesion_1_encoded,hospital_mortality_rate
0,1,0,530001,38.1,132.0,24.0,1,2,2,3,1,1,1,6.5,1,57.0,8.5,2,3.4,1,2209,0.0,5.500000,0.400000,1,0.908885,1.074200,0.727273
1,1,0,533836,37.5,88.0,12.0,1,4,2,1,2,0,2,2.0,0,33.0,64.0,2,2.0,1,2208,1.0,7.333333,0.031250,0,1.256374,1.008410,0.000000
2,1,0,529812,38.3,120.0,28.0,1,5,0,5,2,1,0,3.5,2,37.0,6.4,2,3.4,1,5124,2.0,4.285714,0.531250,1,1.146761,0.997558,0.000000
3,1,0,5262541,37.1,72.0,30.0,0,5,2,1,2,1,2,2.0,1,53.0,7.0,1,3.9,1,2208,2.0,2.400000,0.557143,0,1.146761,0.984256,0.000000
4,0,0,5299629,38.0,52.0,48.0,2,3,0,0,0,1,1,7.0,2,47.0,7.3,1,2.6,0,0,2.0,1.083333,0.356164,1,1.271710,1.860710,0.000000


In [283]:
display(df_encoded2["hospital_mortality_rate"].isnull().sum())

0

In [284]:
### 説明変数と目的変数を定義
df_concat2 = df_encoded2.copy()


# 歪度が大きいカラムを対数変換
# np.log1pを使うことで、log(1+x)を計算する
# df_concat2["respiratory_rate"] = np.log1p(df_concat2["respiratory_rate"])
# df_concat2["total_protein"] = np.log1p(df_concat2["total_protein"])
# df_concat2["abdomo_protein"] = np.log1p(df_concat2["abdomo_protein"])

# "leison_2"のカラムについて、0より大きいなら1、それ以外は0
# df_concat2["lesion_2"] = df_concat2["lesion_2"].apply(lambda x: 1 if x > 0 else 0)
# df_concat2["lesion_2"].value_counts()

# そのままhospital_numberを使うか、hospital_number_encodedを使うかは後で決める
# del df_concat2["hospital_number"]
# del df_concat2["hospital_number_encoded"]

# # そのままlesion_1を使うか、lesion_1_encodedを使うかは後で決める
# del df_concat2["lesion_1"]
# del df_concat2["lesion_1_encoded"]

# del df_concat2["total_protein"]
# del df_concat2["abdomo_protein"]


# 訓練データとテストデータに再度分割
df_train2, df_test2 = df_concat2[:1235], df_concat2[1235:]

# 説明変数
if "outcome" in df_train2.columns:
    x_train2 = df_train2.drop(columns=["outcome"])
else:
    x_train2 = df_train2

y_train2 = df_train2["outcome"]
print("訓練データ", x_train2.shape, y_train2.shape)
# print(y_train2.isnull().sum())

# 目的変数
if "outcome" in df_test2.columns:
    x_test2 = df_test2.drop(columns=["outcome"])
else:
    x_test2 = df_test2
print("テストデータ", x_test2.shape)

訓練データ (1235, 27) (1235,)
テストデータ (824, 27)


In [285]:
display(x_train2.head())
# x_train2.isnull().sum()
x_train2.dtypes

# x_test2.head()

x_train2["ng_reflux_ph_cat"] = x_train2["ng_reflux_ph_cat"].astype("int")
x_test2["ng_reflux_ph_cat"] = x_test2["ng_reflux_ph_cat"].astype("int")

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,mucous_membrane,capillary_refill_time,pain,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,shock_index,protein_ratio,ng_reflux_ph_cat,hospital_number_encoded,lesion_1_encoded,hospital_mortality_rate
0,1,0,530001,38.1,132.0,24.0,1,2,2,3,1,1,1,6.5,1,57.0,8.5,2,3.4,1,2209,5.500000,0.400000,1,0.908885,1.074200,0.727273
1,1,0,533836,37.5,88.0,12.0,1,4,2,1,2,0,2,2.0,0,33.0,64.0,2,2.0,1,2208,7.333333,0.031250,0,1.256374,1.008410,0.000000
2,1,0,529812,38.3,120.0,28.0,1,5,0,5,2,1,0,3.5,2,37.0,6.4,2,3.4,1,5124,4.285714,0.531250,1,1.146761,0.997558,0.000000
3,1,0,5262541,37.1,72.0,30.0,0,5,2,1,2,1,2,2.0,1,53.0,7.0,1,3.9,1,2208,2.400000,0.557143,0,1.146761,0.984256,0.000000
4,0,0,5299629,38.0,52.0,48.0,2,3,0,0,0,1,1,7.0,2,47.0,7.3,1,2.6,0,0,1.083333,0.356164,1,1.271710,1.860710,0.000000


## モデルの学習・予測

#### LightGBM

In [286]:
# lgb_metrics, lgb_imp, lgb_tr_pred, lgb_test_pred = lgb_train_eval(
#     x_train2, y_train2, x_test2, params_lgb1, 5
# )

In [287]:
print("===== LightGBM =====")
print("【マイクロF1スコア】")
print(lgb_metrics)
print(
    "train：{:.2f} ± {:.2f}".format(lgb_metrics[:, 1].mean(), lgb_metrics[:, 1].std())
)
print("val：{:.2f} ± {:.2f}".format(lgb_metrics[:, 2].mean(), lgb_metrics[:, 2].std()))

# print("\n【特徴量重要度】")
display(lgb_imp.sort_values(by="imp_mean", ascending=False))
display(lgb_imp.shape)

# 特徴量重要度が低いものを抽出していく
# low_imp_columns = (
#     lgb_imp[lgb_imp["imp_mean"] < 5]
#     .sort_values(by="imp_mean", ascending=False)
#     .values[:, 0]
# )
# low_imp_columns

===== LightGBM =====
【マイクロF1スコア】
[[0.         0.92712551 0.77732794]
 [1.         0.80364372 0.74089069]
 [2.         0.89271255 0.74493927]
 [3.         0.89068826 0.7611336 ]
 [4.         0.87955466 0.74898785]]
train：0.88 ± 0.04
val：0.75 ± 0.01


,columns,imp_mean,imp_std
9,lesion_1_encoded,824.318620,121.269059
16,pain,636.329430,79.397050
17,protein_ratio,569.361471,105.782113
12,nasogastric_reflux_ph,401.881432,64.975563
7,hospital_number_encoded,355.322873,102.799293
18,pulse,349.033463,40.758234
26,total_protein,346.380098,101.584692
6,hospital_number,276.003669,88.235671
15,packed_cell_volume,222.836947,89.738868
22,shock_index,203.286357,65.453462


(27, 3)

In [288]:
# 提出ファイルの作成
submit = pd.DataFrame(
    data={"id": test_id, "outcome": lgb_test_pred["pred_class_submit"]}
)
# display(submit.head(), submit.shape)
# submit.to_csv("./submit/submit_lgb.csv", index=False)

In [289]:
# # 数値特徴量のリストを取得
# numerical_columns = x_train.select_dtypes(include=["float64", "int64"]).columns
# len(numerical_columns)

# # ヒストグラムを描画
# plt.figure(figsize=(15, 10))
# for i, column in enumerate(numerical_columns, 1):
#     plt.subplot(3, 3, i)  # 3x3のグリッドでプロット
#     sns.histplot(x_train[column], kde=True)
#     plt.title(f"Histogram of {column}")
# plt.tight_layout()
# plt.show()

# # 箱ひげ図を描画
# plt.figure(figsize=(15, 10))
# for i, column in enumerate(numerical_columns, 1):
#     plt.subplot(3, 3, i)
#     sns.boxplot(x=x_train[column])
#     plt.title(f"Boxplot of {column}")
# plt.tight_layout()
# plt.show()

In [290]:
# # 統計量の確認
# stats = x_train[numerical_columns].describe().transpose()

# # 歪度（skewness）を計算
# skewness = x_train[numerical_columns].skew()
# # 結果を表示
# print("統計量:")
# print(stats)
# print("\n歪度:")
# print(skewness)

#### CatBoost

In [ ]:
cat_metrics, cat_imp, cat_tr_pred1, cat_test_pred1 = cat_train_eval(
    x_train1, y_train1, x_test1, category_cols_cat, params_cat1, 5
)

# 別のパラメータを利用
# cat_metrics, cat_imp, cat_tr_pred2, cat_test_pred2 = cat_train_eval(
#     x_train1, y_train1, x_test1, category_cols_cat, params_cat2, 5
# )

# カテゴリ変数のまま使う場合
# cat_metrics, cat_imp, cat_tr_pred3, cat_test_pred3 = cat_train_eval(
#     x_train3, y_train3, x_test3, category_cols_cat, params_cat5, 5
# )

0:	learn: 1.0527168	test: 1.0457868	best: 1.0457868 (0)	total: 17.9ms	remaining: 2m 59s
100:	learn: 0.3662601	test: 0.6291513	best: 0.6291513 (100)	total: 1.37s	remaining: 2m 14s
200:	learn: 0.2255829	test: 0.5965786	best: 0.5963725 (199)	total: 3.17s	remaining: 2m 34s
300:	learn: 0.1522546	test: 0.5920691	best: 0.5898339 (289)	total: 4.65s	remaining: 2m 29s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.589833886
bestIteration = 289

Shrink model to first 290 iterations.
0:	learn: 1.0483613	test: 1.0606161	best: 1.0606161 (0)	total: 30.9ms	remaining: 5m 9s
100:	learn: 0.3306272	test: 0.7090218	best: 0.7090218 (100)	total: 1.68s	remaining: 2m 44s
200:	learn: 0.2031354	test: 0.7157788	best: 0.7039702 (126)	total: 3.27s	remaining: 2m 39s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7039701793
bestIteration = 126

Shrink model to first 127 iterations.
0:	learn: 1.0490114	test: 1.0526478	best: 1.0526478 (0)	total: 14.3ms	remaining: 2m 22s
100:	

In [294]:
print("===== CatBoost =====")
print("【マイクロF1スコア】")
print(cat_metrics)
print(
    "train：{:.2f} ± {:.2f}".format(cat_metrics[:, 1].mean(), cat_metrics[:, 1].std())
)
print("val：{:.2f} ± {:.2f}".format(cat_metrics[:, 2].mean(), cat_metrics[:, 2].std()))

# print("\n【特徴量重要度】")
display(cat_imp.sort_values(by="imp_mean", ascending=False))
display(cat_imp.shape)

===== CatBoost =====
【マイクロF1スコア】
[[0.         0.93825911 0.74898785]
 [1.         0.89979757 0.70850202]
 [2.         0.91801619 0.70040486]
 [3.         0.92813765 0.73279352]
 [4.         0.90587045 0.71255061]]
train：0.92 ± 0.01
val：0.72 ± 0.02


,columns,imp_mean,imp_std
17,pain,9.075625,1.156953
10,lesion_1_encoded,8.988843,0.672291
28,total_protein,7.531238,1.066636
21,rectal_exam_feces,5.167321,0.562306
8,hospital_number_encoded,5.165470,0.476607
14,nasogastric_reflux_ph,4.817158,1.214573
3,abdomo_protein,4.668072,1.257878
2,abdomo_appearance,4.388132,1.281433
20,pulse,3.839090,0.709022
13,nasogastric_reflux,3.721764,0.623358


(29, 3)

In [262]:
# 提出ファイルの作成
submit = pd.DataFrame(
    data={"id": test_id, "outcome": cat_test_pred3["pred_class_submit"]}
)
display(submit.head(), submit.shape)
submit.to_csv("./submit/submit_cat.csv", index=False)

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived


(824, 2)

#### XGBoost

In [202]:
xgb_metrics, xgb_imp, xgb_tr_pred, xgb_test_pred = xgb_train_eval(
    x_train2, y_train2, x_test2, params_xgb1, 5
)

[0]	validation_0-merror:0.12652	validation_1-merror:0.38057
[1]	validation_0-merror:0.09211	validation_1-merror:0.31984
[2]	validation_0-merror:0.08097	validation_1-merror:0.29960
[3]	validation_0-merror:0.07085	validation_1-merror:0.29150
[4]	validation_0-merror:0.06883	validation_1-merror:0.28340
[5]	validation_0-merror:0.06478	validation_1-merror:0.29150
[6]	validation_0-merror:0.05769	validation_1-merror:0.27935
[7]	validation_0-merror:0.05466	validation_1-merror:0.27126
[8]	validation_0-merror:0.05263	validation_1-merror:0.27935
[9]	validation_0-merror:0.05061	validation_1-merror:0.26316
[10]	validation_0-merror:0.04049	validation_1-merror:0.26721
[11]	validation_0-merror:0.03947	validation_1-merror:0.26721
[12]	validation_0-merror:0.03239	validation_1-merror:0.26721
[13]	validation_0-merror:0.03441	validation_1-merror:0.26316
[14]	validation_0-merror:0.02834	validation_1-merror:0.26316
[15]	validation_0-merror:0.02632	validation_1-merror:0.25911
[16]	validation_0-merror:0.02530	v

In [204]:
print("===== XGBoost =====")
print("【マイクロF1スコア】")
print(xgb_metrics)
print(
    "train：{:.2f} ± {:.2f}".format(xgb_metrics[:, 1].mean(), xgb_metrics[:, 1].std())
)
print("val：{:.2f} ± {:.2f}".format(xgb_metrics[:, 2].mean(), xgb_metrics[:, 2].std()))

===== XGBoost =====
【マイクロF1スコア】
[[0.         1.         0.77732794]
 [1.         0.99797571 0.71255061]
 [2.         1.         0.73684211]
 [3.         0.96659919 0.78947368]
 [4.         0.97672065 0.72064777]]
train：0.99 ± 0.01
val：0.75 ± 0.03


In [ ]:
# 提出ファイルの作成
submit = pd.DataFrame(
    data={"id": test_id, "outcome": lgb_test_pred["pred_class_submit"]}
)
# display(submit.head(), submit.shape)
# submit.to_csv("./submit/submit_xgb.csv", index=False)

#### ランダムフォレスト

In [ ]:
rf_metrics, rf_imp, rf_tr_pred, rf_test_pred = rf_train_eval(
    x_train2, y_train2, x_test2, params_rf, 5
)

In [ ]:
print("===== Random Forest =====")
print("【マイクロF1スコア】")
print(rf_metrics)
print("train：{:.2f} ± {:.2f}".format(rf_metrics[:, 1].mean(), rf_metrics[:, 1].std()))
print("val：{:.2f} ± {:.2f}".format(rf_metrics[:, 2].mean(), rf_metrics[:, 2].std()))

print("\n【特徴量重要度】")
print(rf_imp.sort_values(by="imp_mean", ascending=False))

===== Random Forest =====
【マイクロF1スコア】
[[0.         0.93623482 0.74089069]
 [1.         0.93218623 0.70040486]
 [2.         0.93016194 0.70445344]
 [3.         0.93927126 0.71659919]
 [4.         0.93522267 0.71255061]]
train：0.93 ± 0.00
val：0.71 ± 0.01

【特徴量重要度】
                    columns  imp_mean   imp_std
9          lesion_1_encoded  0.103682  0.002828
17            protein_ratio  0.083066  0.003040
26            total_protein  0.076088  0.005102
16                     pain  0.067199  0.005856
12    nasogastric_reflux_ph  0.064818  0.008782
18                    pulse  0.060477  0.002638
6           hospital_number  0.053895  0.003731
2            abdomo_protein  0.051174  0.004244
7   hospital_number_encoded  0.049413  0.003152
15       packed_cell_volume  0.048484  0.003725
22              shock_index  0.035061  0.001046
8                  lesion_1  0.034273  0.003087
5   hospital_mortality_rate  0.032068  0.002519
20              rectal_temp  0.031056  0.002636
21         respir

In [ ]:
# 提出ファイルの作成
submit = pd.DataFrame(
    data={"id": test_id, "outcome": rf_test_pred["pred_class_submit"]}
)
# display(submit.head(), submit.shape)
# submit.to_csv("./submit/submit_rf.csv", index=False)

#### ロジスティック回帰

In [ ]:
lr_metrics, lr_tr_pred, lr_test_pred = lr_train_eval(x_train1, y_train1, x_test1, 3, 5)

In [ ]:
print("===== Logitstic Regression =====")
print("【マイクロF1スコア】")
print(lr_metrics)
print("train：{:.2f} ± {:.2f}".format(lr_metrics[:, 1].mean(), lr_metrics[:, 1].std()))
print("val：{:.2f} ± {:.2f}".format(lr_metrics[:, 2].mean(), lr_metrics[:, 2].std()))
# print("\n【特徴量重要度】")
# print(lr_imp.sort_values(by="imp_mean", ascending=False))

===== Logitstic Regression =====
【マイクロF1スコア】
[[0.         0.72064777 0.74089069]
 [1.         0.73582996 0.71255061]
 [2.         0.73380567 0.72469636]
 [3.         0.72773279 0.71255061]
 [4.         0.73178138 0.67611336]]
train：0.73 ± 0.01
val：0.71 ± 0.02


#### Oputuna(LightGBM)

In [ ]:
x_train = x_train1
y_train = y_train1
x_test = x_test1

sampler = optuna.samplers.TPESampler(seed=123)
study1 = optuna.create_study(sampler=sampler, direction="maximize")
study1.optimize(objective_lgb, n_trials=30)

[I 2025-03-30 14:26:39,978] A new study created in memory with name: no-name-5367cfe5-c5bd-43c4-a0c4-bd9583dc86ee


Training until validation scores don't improve for 100 rounds


/tmp/ipykernel_396/926144997.py:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "min_gain_to_split": trial.suggest_loguniform("min_gain_to_split", 1e-8, 1.0),


[100]	training's multi_logloss: 0.26796	training's micro_f1: 0.940283	valid_1's multi_logloss: 0.583814	valid_1's micro_f1: 0.769231
[200]	training's multi_logloss: 0.124041	training's micro_f1: 0.998988	valid_1's multi_logloss: 0.604526	valid_1's micro_f1: 0.785425
Early stopping, best iteration is:
[108]	training's multi_logloss: 0.251517	training's micro_f1: 0.947368	valid_1's multi_logloss: 0.580627	valid_1's micro_f1: 0.773279
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.238962	training's micro_f1: 0.948381	valid_1's multi_logloss: 0.747394	valid_1's micro_f1: 0.712551
Early stopping, best iteration is:
[24]	training's multi_logloss: 0.53324	training's micro_f1: 0.804656	valid_1's multi_logloss: 0.706004	valid_1's micro_f1: 0.736842
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.253948	training's micro_f1: 0.94332	valid_1's multi_logloss: 0.673581	valid_1's micro_f1: 0.704453
Early 

[I 2025-03-30 14:26:56,583] Trial 0 finished with value: 0.7449392712550607 and parameters: {'min_child_samples': 37, 'max_depth': 5, 'bagging_fraction': 0.65, 'feature_fraction': 0.8, 'min_gain_to_split': 0.005698384608345678}. Best is trial 0 with value: 0.7449392712550607.


...

[100]	training's multi_logloss: 0.187722	training's micro_f1: 0.98583	valid_1's multi_logloss: 0.72345	valid_1's micro_f1: 0.708502
Early stopping, best iteration is:
[26]	training's multi_logloss: 0.478582	training's micro_f1: 0.838057	valid_1's multi_logloss: 0.684002	valid_1's micro_f1: 0.740891
Training until validation scores don't improve for 100 rounds


/tmp/ipykernel_396/926144997.py:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "min_gain_to_split": trial.suggest_loguniform("min_gain_to_split", 1e-8, 1.0),


[100]	training's multi_logloss: 0.257706	training's micro_f1: 0.945344	valid_1's multi_logloss: 0.573774	valid_1's micro_f1: 0.748988
Early stopping, best iteration is:
[55]	training's multi_logloss: 0.389111	training's micro_f1: 0.883603	valid_1's multi_logloss: 0.591619	valid_1's micro_f1: 0.761134
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.237334	training's micro_f1: 0.953441	valid_1's multi_logloss: 0.746692	valid_1's micro_f1: 0.704453
Early stopping, best iteration is:
[25]	training's multi_logloss: 0.507483	training's micro_f1: 0.804656	valid_1's multi_logloss: 0.712653	valid_1's micro_f1: 0.724696
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.244446	training's micro_f1: 0.948381	valid_1's multi_logloss: 0.68757	valid_1's micro_f1: 0.708502
Early stopping, best iteration is:
[57]	training's multi_logloss: 0.359517	training's micro_f1: 0.882591	valid_1's multi_logloss: 0.660501	

[I 2025-03-30 14:28:54,617] Trial 29 finished with value: 0.7376518218623481 and parameters: {'min_child_samples': 22, 'max_depth': 4, 'bagging_fraction': 0.7, 'feature_fraction': 0.75, 'min_gain_to_split': 0.0439273148314347}. Best is trial 28 with value: 0.7489878542510121.


[100]	training's multi_logloss: 0.235497	training's micro_f1: 0.960526	valid_1's multi_logloss: 0.729844	valid_1's micro_f1: 0.692308
Early stopping, best iteration is:
[22]	training's multi_logloss: 0.538156	training's micro_f1: 0.815789	valid_1's multi_logloss: 0.702208	valid_1's micro_f1: 0.728745


In [191]:
# 結果
trial1 = study1.best_trial
print("マイクロF1スコア(best)：{:.4f}".format(trial1.value))
print("ベストパラメータ：{}".format(trial1.params))

マイクロF1スコア(best)：0.7490
ベストパラメータ：{'min_child_samples': 24, 'max_depth': 5, 'bagging_fraction': 0.7, 'feature_fraction': 0.6, 'min_gain_to_split': 0.015142971393036024}


In [192]:
params_best_lgb = trial1.params
params_best_lgb.update(params_base_lgb)  # ベースパラメータを追加
display(params_best_lgb)

{'min_child_samples': 24,
 'max_depth': 5,
 'bagging_fraction': 0.7,
 'feature_fraction': 0.6,
 'min_gain_to_split': 0.015142971393036024,
 'boosting_type': 'gbdt',
 'objective': 'multiclass',
 'num_class': 3,
 'metric': 'multi_logloss',
 'importance_type': 'gain',
 'random_state': 42,
 'bagging_freq': 1,
 'n_estimators': 20000,
 'learning_rate': 0.1,
 'lambda_l1': 0.0,
 'lambda_l2': 1.0,
 'verbose': -1}

In [ ]:
lgb_metrics_oputuna, lgb_imp_oputuna, lgb_tr_pred_oputuna, lgb_test_pred_oputuna = (
    lgb_train_eval(x_train1, y_train1, x_test1, params_best_lgb, 5)
)

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.272128	training's micro_f1: 0.935223	valid_1's multi_logloss: 0.590891	valid_1's micro_f1: 0.777328
Early stopping, best iteration is:
[87]	training's multi_logloss: 0.30129	training's micro_f1: 0.919028	valid_1's multi_logloss: 0.593188	valid_1's micro_f1: 0.785425
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.242734	training's micro_f1: 0.947368	valid_1's multi_logloss: 0.753366	valid_1's micro_f1: 0.708502
Early stopping, best iteration is:
[30]	training's multi_logloss: 0.481244	training's micro_f1: 0.819838	valid_1's multi_logloss: 0.7155	valid_1's micro_f1: 0.700405
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.253322	training's micro_f1: 0.955466	valid_1's multi_logloss: 0.684442	valid_1's micro_f1: 0.712551
Early stopping, best iteration is:
[54]	training's multi_logloss: 0.3825	traini

In [237]:
print("===== LightGBM =====")
print("【マイクロF1スコア】")
print(lgb_metrics_oputuna)
print(
    "train：{:.2f} ± {:.2f}".format(
        lgb_metrics_oputuna[:, 1].mean(), lgb_metrics_oputuna[:, 1].std()
    )
)
print(
    "val：{:.2f} ± {:.2f}".format(
        lgb_metrics_oputuna[:, 2].mean(), lgb_metrics_oputuna[:, 2].std()
    )
)

print("\n【特徴量重要度】")
display(lgb_imp_oputuna.sort_values(by="imp_mean", ascending=False).head())
display(lgb_imp_oputuna.shape)

# 特徴量重要度が低いものを抽出していく
# low_imp_oputuna_columns = (
#     lgb_imp_oputuna[lgb_imp_oputuna["imp_mean"] < 5]
#     .sort_values(by="imp_mean", ascending=False)
#     .values[:, 0]
# )
# low_imp_oputuna_columns

===== LightGBM =====
【マイクロF1スコア】


NameError: name 'lgb_metrics_oputuna' is not defined

In [200]:
# 提出ファイルの作成
submit = pd.DataFrame(
    data={"id": test_id, "outcome": lgb_test_pred_oputuna["pred_class_submit"]}
)
display(submit.head(), submit.shape)
# submit.to_csv("./submit/submit_lgb_oputuna.csv", index=False)

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived


(824, 2)

In [201]:
# display(lgb_tr_pred_oputuna.head())
display(lgb_test_pred_oputuna.head())

,クラス_0_lgb,クラス_1_lgb,クラス_2_lgb,pred_class,pred_class_submit
0,0.037069,0.026946,0.935985,2,lived
1,0.744385,0.072501,0.183114,0,died
2,0.169182,0.039770,0.791048,2,lived
3,0.056269,0.836224,0.107508,1,euthanized
4,0.200404,0.052343,0.747253,2,lived


#### Oputuna(CatBoost)

In [251]:
# ラベルエンコーディングしたデータセット
# x_train = x_train1
# y_train = y_train1
# x_test = x_test1

# カテゴリ変数のままのデータセット
x_train = x_train3
y_train = y_train3
x_test = x_test3
categorical_features = category_cols_cat

sampler = optuna.samplers.TPESampler(seed=123)
study3 = optuna.create_study(sampler=sampler, direction="maximize")
study3.optimize(objective_cat, n_trials=30)

[I 2025-04-03 14:45:31,703] A new study created in memory with name: no-name-fed9585d-b0bc-4877-9986-5461047b8649


0:	learn: 1.0520308	test: 1.0504271	best: 1.0504271 (0)	total: 8.07ms	remaining: 1m 20s
100:	learn: 0.5024619	test: 0.6039279	best: 0.6039279 (100)	total: 794ms	remaining: 1m 17s
200:	learn: 0.4063381	test: 0.5803280	best: 0.5801097 (199)	total: 1.55s	remaining: 1m 15s
300:	learn: 0.3348063	test: 0.5789338	best: 0.5772510 (273)	total: 2.51s	remaining: 1m 20s
400:	learn: 0.2758379	test: 0.5835651	best: 0.5770127 (333)	total: 3.04s	remaining: 1m 12s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.5770127196
bestIteration = 333

Shrink model to first 334 iterations.
0:	learn: 1.0463097	test: 1.0576947	best: 1.0576947 (0)	total: 6.89ms	remaining: 1m 8s
100:	learn: 0.4719447	test: 0.7090574	best: 0.7090260 (99)	total: 676ms	remaining: 1m 6s
200:	learn: 0.3804056	test: 0.7094294	best: 0.7048270 (143)	total: 1.24s	remaining: 1m
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7048269548
bestIteration = 143

Shrink model to first 144 iterations.
0:	lear

[I 2025-04-03 14:45:42,381] Trial 0 finished with value: 0.7336032388663968 and parameters: {'learning_rate': 0.10684874790620232, 'depth': 5, 'bagging_temperature': 0.2268514535642031, 'border_count': 155, 'min_data_in_leaf': 36, 'colsample_bylevel': 0.7115532300622305}. Best is trial 0 with value: 0.7336032388663968.


200:	learn: 0.3752247	test: 0.6744522	best: 0.6710873 (111)	total: 1.07s	remaining: 52.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6710873465
bestIteration = 111

Shrink model to first 112 iterations.
0:	learn: 0.9531230	test: 0.9564742	best: 0.9564742 (0)	total: 28.4ms	remaining: 4m 44s
100:	learn: 0.1486896	test: 0.6221568	best: 0.6212649 (93)	total: 2.05s	remaining: 3m 20s
200:	learn: 0.0652993	test: 0.6096192	best: 0.6090149 (199)	total: 3.91s	remaining: 3m 10s
300:	learn: 0.0378155	test: 0.6169813	best: 0.6077682 (216)	total: 6.37s	remaining: 3m 25s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6077681963
bestIteration = 216

Shrink model to first 217 iterations.
0:	learn: 0.9706708	test: 0.9954919	best: 0.9954919 (0)	total: 12.1ms	remaining: 2m
100:	learn: 0.1305575	test: 0.7362041	best: 0.6992454 (59)	total: 1.63s	remaining: 2m 39s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6992453866
bestIteration = 59

[I 2025-04-03 14:46:01,103] Trial 1 finished with value: 0.7174089068825911 and parameters: {'learning_rate': 0.281000856274069, 'depth': 8, 'bagging_temperature': 0.48093190148436094, 'border_count': 119, 'min_data_in_leaf': 18, 'colsample_bylevel': 0.8645248536920208}. Best is trial 0 with value: 0.7336032388663968.


...

Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7039806137
bestIteration = 134

Shrink model to first 135 iterations.
0:	learn: 0.9924264	test: 0.9921656	best: 0.9921656 (0)	total: 12.3ms	remaining: 2m 2s
100:	learn: 0.1748146	test: 0.6186581	best: 0.6186581 (100)	total: 1.91s	remaining: 3m 7s
200:	learn: 0.0745659	test: 0.6256340	best: 0.6146959 (138)	total: 4.02s	remaining: 3m 15s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6146958711
bestIteration = 138

Shrink model to first 139 iterations.
0:	learn: 0.9806446	test: 1.0131307	best: 1.0131307 (0)	total: 11.3ms	remaining: 1m 52s
100:	learn: 0.1876086	test: 0.7525792	best: 0.7323277 (66)	total: 1.84s	remaining: 3m
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7323276726
bestIteration = 66

Shrink model to first 67 iterations.
0:	learn: 0.9911334	test: 1.0104701	best: 1.0104701 (0)	total: 33.7ms	remaining: 5m 37s
100:	learn: 0.1656052	test: 0.6739878	best: 0.6710309 (

[I 2025-04-03 15:05:01,756] Trial 28 finished with value: 0.719838056680162 and parameters: {'learning_rate': 0.24421352885634778, 'depth': 8, 'bagging_temperature': 0.31100349745346656, 'border_count': 177, 'min_data_in_leaf': 35, 'colsample_bylevel': 0.7254642401072012}. Best is trial 12 with value: 0.7392712550607288.


Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6905914458
bestIteration = 40

Shrink model to first 41 iterations.
0:	learn: 1.0471816	test: 1.0409698	best: 1.0409698 (0)	total: 3.79ms	remaining: 37.9s
100:	learn: 0.5124545	test: 0.6364183	best: 0.6364183 (100)	total: 703ms	remaining: 1m 8s
200:	learn: 0.4196403	test: 0.6153453	best: 0.6153122 (197)	total: 1.35s	remaining: 1m 6s
300:	learn: 0.3437482	test: 0.5960546	best: 0.5960546 (300)	total: 2.02s	remaining: 1m 5s
400:	learn: 0.2883295	test: 0.5908093	best: 0.5905892 (398)	total: 2.69s	remaining: 1m 4s
500:	learn: 0.2477022	test: 0.5852669	best: 0.5843545 (495)	total: 3.37s	remaining: 1m 3s
600:	learn: 0.2107431	test: 0.5827001	best: 0.5824944 (523)	total: 3.92s	remaining: 1m 1s
700:	learn: 0.1865791	test: 0.5849263	best: 0.5823048 (602)	total: 4.48s	remaining: 59.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.5823048424
bestIteration = 602

Shrink model to first 603 iterations.
0:	lear

[I 2025-04-03 15:05:25,988] Trial 29 finished with value: 0.7336032388663968 and parameters: {'learning_rate': 0.09874604531692224, 'depth': 5, 'bagging_temperature': 0.22130948311825735, 'border_count': 157, 'min_data_in_leaf': 26, 'colsample_bylevel': 0.6256498498463319}. Best is trial 12 with value: 0.7392712550607288.


Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6858278028
bestIteration = 167

Shrink model to first 168 iterations.


In [252]:
# 結果
trial3 = study3.best_trial
print("マイクロF1スコア(best)：{:.4f}".format(trial3.value))
print("ベストパラメータ：{}".format(trial3.params))

マイクロF1スコア(best)：0.7393
ベストパラメータ：{'learning_rate': 0.10835456427924503, 'depth': 3, 'bagging_temperature': 0.23836663646047773, 'border_count': 175, 'min_data_in_leaf': 50, 'colsample_bylevel': 0.6657674216871736}


In [255]:
params_best_cat = trial3.params
params_best_cat.update(params_base_cat)  # ベースパラメータを追加
display(params_best_cat)

{'learning_rate': 0.10835456427924503,
 'depth': 3,
 'bagging_temperature': 0.23836663646047773,
 'border_count': 175,
 'min_data_in_leaf': 50,
 'colsample_bylevel': 0.6657674216871736,
 'loss_function': 'MultiClass',
 'eval_metric': 'MultiClass',
 'iterations': 10000,
 'early_stopping_rounds': 100,
 'random_state': 123,
 'verbose': 0}

In [254]:
cat_metrics_oputuna, cat_imp_oputuna, cat_tr_pred_oputuna, cat_test_pred_oputuna = (
    cat_train_eval(x_train1, y_train1, x_test1, category_cols_cat, params_best_cat, 5)
)

0:	learn: 1.0564001	test: 1.0508564	best: 1.0508564 (0)	total: 3.86ms	remaining: 38.6s
100:	learn: 0.5939154	test: 0.6362278	best: 0.6362278 (100)	total: 332ms	remaining: 32.5s
200:	learn: 0.5309141	test: 0.6151706	best: 0.6151483 (196)	total: 608ms	remaining: 29.7s
300:	learn: 0.4849880	test: 0.6040168	best: 0.6038479 (298)	total: 896ms	remaining: 28.9s
400:	learn: 0.4434806	test: 0.5917459	best: 0.5916871 (399)	total: 1.17s	remaining: 28.1s
500:	learn: 0.4108397	test: 0.5885811	best: 0.5885811 (500)	total: 1.44s	remaining: 27.4s
600:	learn: 0.3813808	test: 0.5914563	best: 0.5885484 (501)	total: 1.76s	remaining: 27.5s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.5885483731
bestIteration = 501

Shrink model to first 502 iterations.
0:	learn: 1.0409457	test: 1.0463637	best: 1.0463637 (0)	total: 2.23ms	remaining: 22.3s
100:	learn: 0.5608094	test: 0.6964825	best: 0.6964825 (100)	total: 309ms	remaining: 30.3s
200:	learn: 0.5001692	test: 0.6889179	best: 0.6889179 (20

KeyboardInterrupt: 

In [736]:
print("===== CatBoost =====")
print("【マイクロF1スコア】")
print(cat_metrics_oputuna)
print(
    "train：{:.2f} ± {:.2f}".format(
        cat_metrics_oputuna[:, 1].mean(), cat_metrics_oputuna[:, 1].std()
    )
)
print(
    "val：{:.2f} ± {:.2f}".format(
        cat_metrics_oputuna[:, 2].mean(), cat_metrics_oputuna[:, 2].std()
    )
)

print("\n【特徴量重要度】")
display(cat_imp_oputuna.sort_values(by="imp_mean", ascending=False).head())
display(cat_imp_oputuna.shape)

# 特徴量重要度が低いものを抽出していく
# low_imp_oputuna_columns = (
#     cat_imp_oputuna[cat_imp_oputuna["imp_mean"] < 5]
#     .sort_values(by="imp_mean", ascending=False)
#     .values[:, 0]
# )
# low_imp_oputuna_columns

===== CatBoost =====
【マイクロF1スコア】
[[0.         0.99898785 0.7611336 ]
 [1.         0.97064777 0.71659919]
 [2.         0.99190283 0.68825911]
 [3.         0.98987854 0.75303644]
 [4.         0.98178138 0.71659919]]
train：0.99 ± 0.01
val：0.73 ± 0.03

【特徴量重要度】


,columns,imp_mean,imp_std
10,lesion_1_encoded,10.247233,0.384534
28,total_protein,7.912146,1.038748
17,pain,7.799261,0.883287
14,nasogastric_reflux_ph,6.419794,0.484610
8,hospital_number_encoded,5.931255,0.589149


(29, 3)

In [737]:
# 提出ファイルの作成
submit = pd.DataFrame(
    data={"id": test_id, "outcome": cat_test_pred_oputuna["pred_class_submit"]}
)
display(submit.head(), submit.shape)
submit.to_csv("./submit/submit_cat_oputuna.csv", index=False)

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived


(824, 2)

In [ ]:
# display(lgb_tr_pred_oputuna.head())
display(lgb_test_pred_oputuna.head())

,クラス_0_lgb,クラス_1_lgb,クラス_2_lgb,pred_class,pred_class_submit
0,0.037069,0.026946,0.935985,2,lived
1,0.744385,0.072501,0.183114,0,died
2,0.169182,0.039770,0.791048,2,lived
3,0.056269,0.836224,0.107508,1,euthanized
4,0.200404,0.052343,0.747253,2,lived


#### Oputuna(XGBoost)

In [328]:
x_train = x_train1
y_train = y_train1
x_test = x_test1

sampler = optuna.samplers.TPESampler(seed=123)
study2 = optuna.create_study(sampler=sampler, direction="maximize")
study2.optimize(objective_xgb, n_trials=30)

[I 2025-03-31 17:50:53,029] A new study created in memory with name: no-name-2ee5eea5-8bf8-47bb-a74d-3242cfe4e986


[0]	validation_0-merror:0.25506	validation_1-merror:0.31174
[1]	validation_0-merror:0.23684	validation_1-merror:0.27530
[2]	validation_0-merror:0.23583	validation_1-merror:0.25911
[3]	validation_0-merror:0.22976	validation_1-merror:0.29150
[4]	validation_0-merror:0.22773	validation_1-merror:0.26316


/tmp/ipykernel_406/450742611.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "eta": trial.suggest_loguniform("eta", 0.005, 0.3),
/tmp/ipykernel_406/450742611.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.1, 10),
/tmp/ipykernel_406/450742611.py:35: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "gamma": trial.suggest_loguniform("gamma", 1e-8, 1.0),


[5]	validation_0-merror:0.21964	validation_1-merror:0.27935
[6]	validation_0-merror:0.21964	validation_1-merror:0.27126
[7]	validation_0-merror:0.21862	validation_1-merror:0.27530
[8]	validation_0-merror:0.21964	validation_1-merror:0.27126
[9]	validation_0-merror:0.21761	validation_1-merror:0.25911
[10]	validation_0-merror:0.21660	validation_1-merror:0.26721
[11]	validation_0-merror:0.20850	validation_1-merror:0.25911
[12]	validation_0-merror:0.20850	validation_1-merror:0.25911
[13]	validation_0-merror:0.20951	validation_1-merror:0.25911
[14]	validation_0-merror:0.20648	validation_1-merror:0.25506
[15]	validation_0-merror:0.20749	validation_1-merror:0.26316
[16]	validation_0-merror:0.21053	validation_1-merror:0.24696
[17]	validation_0-merror:0.20547	validation_1-merror:0.24291
[18]	validation_0-merror:0.20547	validation_1-merror:0.25101
[19]	validation_0-merror:0.20243	validation_1-merror:0.24696
[20]	validation_0-merror:0.20344	validation_1-merror:0.25506
[21]	validation_0-merror:0.20

[I 2025-03-31 17:51:02,082] Trial 0 finished with value: 0.7433198380566802 and parameters: {'eta': 0.08657600816771709, 'min_child_weight': 0.37348973478010355, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.85, 'gamma': 2.4257815076675944e-05}. Best is trial 0 with value: 0.7433198380566802.


...

[I 2025-03-31 17:56:58,109] Trial 28 finished with value: 0.7392712550607289 and parameters: {'eta': 0.03258481384536941, 'min_child_weight': 0.33844902209355243, 'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.85, 'gamma': 0.00026361890699201326}. Best is trial 18 with value: 0.7506072874493926.


[0]	validation_0-merror:0.23684	validation_1-merror:0.32389
[1]	validation_0-merror:0.17105	validation_1-merror:0.29150
[2]	validation_0-merror:0.15992	validation_1-merror:0.27935
[3]	validation_0-merror:0.15486	validation_1-merror:0.27935
[4]	validation_0-merror:0.13360	validation_1-merror:0.27530
[5]	validation_0-merror:0.12652	validation_1-merror:0.26721
[6]	validation_0-merror:0.11640	validation_1-merror:0.25506
[7]	validation_0-merror:0.11134	validation_1-merror:0.25911
[8]	validation_0-merror:0.10425	validation_1-merror:0.25506
[9]	validation_0-merror:0.09818	validation_1-merror:0.26721
[10]	validation_0-merror:0.08300	validation_1-merror:0.26721
[11]	validation_0-merror:0.08097	validation_1-merror:0.26316
[12]	validation_0-merror:0.07895	validation_1-merror:0.26721
[13]	validation_0-merror:0.07085	validation_1-merror:0.26316
[14]	validation_0-merror:0.05972	validation_1-merror:0.26316
[15]	validation_0-merror:0.05668	validation_1-merror:0.26721
[16]	validation_0-merror:0.04656	v

/tmp/ipykernel_406/450742611.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "eta": trial.suggest_loguniform("eta", 0.005, 0.3),
/tmp/ipykernel_406/450742611.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.1, 10),
/tmp/ipykernel_406/450742611.py:35: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "gamma": trial.suggest_loguniform("gamma", 1e-8, 1.0),


[25]	validation_0-merror:0.01113	validation_1-merror:0.25911
[26]	validation_0-merror:0.01113	validation_1-merror:0.25101
[27]	validation_0-merror:0.00810	validation_1-merror:0.25911
[28]	validation_0-merror:0.00709	validation_1-merror:0.26721
[29]	validation_0-merror:0.00506	validation_1-merror:0.25101
[30]	validation_0-merror:0.00304	validation_1-merror:0.24291
[31]	validation_0-merror:0.00304	validation_1-merror:0.24696
[32]	validation_0-merror:0.00202	validation_1-merror:0.25101
[33]	validation_0-merror:0.00202	validation_1-merror:0.24696
[34]	validation_0-merror:0.00202	validation_1-merror:0.25101
[35]	validation_0-merror:0.00202	validation_1-merror:0.25101
[36]	validation_0-merror:0.00304	validation_1-merror:0.25506
[37]	validation_0-merror:0.00304	validation_1-merror:0.26316
[38]	validation_0-merror:0.00202	validation_1-merror:0.26316
[39]	validation_0-merror:0.00202	validation_1-merror:0.25911
[40]	validation_0-merror:0.00101	validation_1-merror:0.25506
[41]	validation_0-merror

[I 2025-03-31 17:57:02,933] Trial 29 finished with value: 0.7295546558704453 and parameters: {'eta': 0.27911760925712076, 'min_child_weight': 2.8329892219431816, 'max_depth': 7, 'subsample': 0.65, 'colsample_bytree': 0.75, 'gamma': 2.204000615038555e-05}. Best is trial 18 with value: 0.7506072874493926.


In [329]:
# 結果
trial2 = study2.best_trial
print("マイクロF1スコア(best)：{:.4f}".format(trial2.value))
print("ベストパラメータ：{}".format(trial2.params))

マイクロF1スコア(best)：0.7506
ベストパラメータ：{'eta': 0.1458045281501414, 'min_child_weight': 0.45455533258130515, 'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.030446090060040447}


In [333]:
params_best_xgb = trial2.params
params_best_xgb.update(params_base_xgb)  # ベースパラメータを追加
display(params_best_xgb)

{'eta': 0.1458045281501414,
 'min_child_weight': 0.45455533258130515,
 'max_depth': 6,
 'subsample': 0.8,
 'colsample_bytree': 0.8,
 'gamma': 0.030446090060040447,
 'objective': 'multi:softprob',
 'num_class': 3,
 'booster': 'gbtree',
 'eval_metric': 'merror',
 'random_state': 123,
 'importance_type': 'total_gain',
 'early_stopping_rounds': 100,
 'n_estimators': 10000,
 'alpha': 0.0,
 'lambda': 1.0}

In [334]:
xgb_metrics_oputuna, xgb_imp, xgb_tr_pred_oputuna, xgb_test_pred_oputuna = (
    xgb_train_eval(x_train, y_train, x_test, params_best_xgb, 5)
)

[0]	validation_0-merror:0.12652	validation_1-merror:0.38057
[1]	validation_0-merror:0.09211	validation_1-merror:0.31984
[2]	validation_0-merror:0.08097	validation_1-merror:0.29960
[3]	validation_0-merror:0.07085	validation_1-merror:0.29150
[4]	validation_0-merror:0.06883	validation_1-merror:0.28340
[5]	validation_0-merror:0.06478	validation_1-merror:0.29150
[6]	validation_0-merror:0.05769	validation_1-merror:0.27935
[7]	validation_0-merror:0.05466	validation_1-merror:0.27126
[8]	validation_0-merror:0.05263	validation_1-merror:0.27935
[9]	validation_0-merror:0.05061	validation_1-merror:0.26316
[10]	validation_0-merror:0.04049	validation_1-merror:0.26721
[11]	validation_0-merror:0.03947	validation_1-merror:0.26721
[12]	validation_0-merror:0.03239	validation_1-merror:0.26721
[13]	validation_0-merror:0.03441	validation_1-merror:0.26316
[14]	validation_0-merror:0.02834	validation_1-merror:0.26316
[15]	validation_0-merror:0.02632	validation_1-merror:0.25911
[16]	validation_0-merror:0.02530	v

In [335]:
print("===== XGBoost =====")
print("【マイクロF1スコア】")
print(xgb_metrics_oputuna)
print(
    "train：{:.2f} ± {:.2f}".format(
        xgb_metrics_oputuna[:, 1].mean(), xgb_metrics_oputuna[:, 1].std()
    )
)
print(
    "val：{:.2f} ± {:.2f}".format(
        xgb_metrics_oputuna[:, 2].mean(), xgb_metrics_oputuna[:, 2].std()
    )
)

===== XGBoost =====
【マイクロF1スコア】
[[0.         1.         0.77732794]
 [1.         0.99797571 0.71255061]
 [2.         1.         0.73684211]
 [3.         0.96659919 0.78947368]
 [4.         0.97672065 0.72064777]]
train：0.99 ± 0.01
val：0.75 ± 0.03


## アンサンブル・スタッキング

#### アンサンブル(1段階目)

In [ ]:
### 最高スコア → lgb, catの2つのモデルを使った平均値

# w1 = 0.24836154199336424
# w2 = 0.47218922277282843
# w3 = 0.2794492352338073

# ensemble_mean_train1 = pd.DataFrame(
#     w1 * lgb_tr_pred[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
#     + w2 * xgb_tr_pred[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
#     + w3 * cat_tr_pred1[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     # + rf_tr_pred[["クラス_0_rf", "クラス_1_rf", "クラス_2_rf"]].values
# )
# ensemble_mean_train1["pred_class"] = ensemble_mean_train1.idxmax(axis=1)

# ensemble_mean_test1 = pd.DataFrame(
#     w1 * lgb_test_pred[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
#     + w2 * xgb_test_pred[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
#     + w3 * cat_test_pred1[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     # + rf_test_pred[["クラス_0_rf", "クラス_1_rf", "クラス_2_rf"]].values
# )
# ensemble_mean_test1["pred_class"] = ensemble_mean_test1.idxmax(axis=1)

# CatBoost(カテゴリ変数のままのデータセット)
# w1 = 0.13181231154985498
# w2 = 0.4251279266197578
# w3 = 0.4430597618303871
# ensemble_mean_train1 = pd.DataFrame(
#     w1 * lgb_tr_pred[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
#     + w2 * xgb_tr_pred[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
#     + w3 * cat_tr_pred3[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     # + rf_tr_pred[["クラス_0_rf", "クラス_1_rf", "クラス_2_rf"]].values
# )
# ensemble_mean_train1["pred_class"] = ensemble_mean_train1.idxmax(axis=1)

# ensemble_mean_test1 = pd.DataFrame(
#     w1 * lgb_test_pred[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
#     + w2 * xgb_test_pred[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
#     + w3 * cat_test_pred3[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     # + rf_test_pred[["クラス_0_rf", "クラス_1_rf", "クラス_2_rf"]].values
# )
# ensemble_mean_test1["pred_class"] = ensemble_mean_test1.idxmax(axis=1)


# ensemble_mean_train1 = (
#     pd.DataFrame(
#         lgb_tr_pred[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
#         # xgb_tr_pred[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
#         + cat_tr_pred[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     )
#     / 2
# )
# ensemble_mean_train1["pred_class"] = ensemble_mean_train1.idxmax(axis=1)

# ensemble_mean_test1 = (
#     pd.DataFrame(
#         lgb_test_pred[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
#         # xgb_test_pred[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
#         + cat_test_pred[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     )
#     / 2
# )
# ensemble_mean_test1["pred_class"] = ensemble_mean_test1.idxmax(axis=1)

# # CatBoost 別のパラメータ同士
# w1 = 0.15610619746660093
# w2 = 0.8438938025333991
# ensemble_mean_train1 = pd.DataFrame(
#     w1 * cat_tr_pred1[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     + w2 * cat_tr_pred2[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
# )
# ensemble_mean_train1["pred_class"] = ensemble_mean_train1.idxmax(axis=1)

# ensemble_mean_test1 = pd.DataFrame(
#     w1 * cat_test_pred1[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     + w2 * cat_test_pred2[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
# )
# ensemble_mean_test1["pred_class"] = ensemble_mean_test1.idxmax(axis=1)

# CatBoost 別のパラメータ同士(3つ)
# w1 = 0.26107783056561423
# w2 = 0.04727583610906832
# w3 = 0.6916463333253174
# ensemble_mean_train1 = pd.DataFrame(
#     w1 * cat_tr_pred1[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     + w2 * cat_tr_pred2[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     + w3 * cat_tr_pred3[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
# )
# ensemble_mean_train1["pred_class"] = ensemble_mean_train1.idxmax(axis=1)

# ensemble_mean_test1 = pd.DataFrame(
#     w1 * cat_test_pred1[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     + w2 * cat_test_pred2[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
#     + w3 * cat_test_pred3[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
# )
# ensemble_mean_test1["pred_class"] = ensemble_mean_test1.idxmax(axis=1)


# ラベルを元に戻すための辞書
label_map = {0: "died", 1: "euthanized", 2: "lived"}

ensemble_mean_train1["pred_class_submit"] = ensemble_mean_train1["pred_class"].map(
    label_map
)
ensemble_mean_test1["pred_class_submit"] = ensemble_mean_test1["pred_class"].map(
    label_map
)

# 精度を確認
print(
    "マイクロF1スコア:{:.5f}".format(
        f1_score(y_train1, ensemble_mean_train1["pred_class"], average="micro")
    )
)

マイクロF1スコア:0.74170


In [275]:
ensemble_mean_test1.head()

,0,1,2,pred_class,pred_class_submit
0,0.033282,0.034609,0.932109,2,lived
1,0.719837,0.096145,0.184018,0,died
2,0.120194,0.061145,0.818661,2,lived
3,0.050305,0.857578,0.092117,1,euthanized
4,0.145604,0.051514,0.802882,2,lived


In [276]:
# 提出ファイルの作成
submit = pd.DataFrame(
    data={"id": test_id, "outcome": ensemble_mean_test1["pred_class_submit"]}
)
# display(submit.head(), submit.shape, submit.isnull().sum())

submit.to_csv("./submit/submit_ensemble_mean2.csv", index=False)
# submit.to_csv("./submit/submit_ensemble_cat.csv", index=False)

In [287]:
### Optunaを使ったアンサンブルのweightの最適化
# model_predict1 = lgb_tr_pred[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
# model_predict2 = xgb_tr_pred[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
# model_predict3 = cat_tr_pred3[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
# y_true = y_train1

model_predict1 = cat_tr_pred1[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
model_predict2 = cat_tr_pred2[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
# model_predict3 = cat_tr_pred3[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
y_true = y_train1

sampler = optuna.samplers.TPESampler(seed=123)
stady9 = 0
study9 = optuna.create_study(sampler=sampler, direction="maximize")
study9.optimize(objective_ensemble_weight, n_trials=100)

[I 2025-04-03 15:30:39,928] A new study created in memory with name: no-name-aadda8f8-a230-483f-af2e-ab64fc56291f
[I 2025-04-03 15:30:39,934] Trial 0 finished with value: 0.7246963562753036 and parameters: {'w1': 0.6964691855978616, 'w2': 0.28613933495037946}. Best is trial 0 with value: 0.7246963562753036.
[I 2025-04-03 15:30:39,938] Trial 1 finished with value: 0.7360323886639676 and parameters: {'w1': 0.2268514535642031, 'w2': 0.5513147690828912}. Best is trial 1 with value: 0.7360323886639676.
[I 2025-04-03 15:30:39,943] Trial 2 finished with value: 0.7271255060728745 and parameters: {'w1': 0.7194689697855631, 'w2': 0.42310646012446096}. Best is trial 1 with value: 0.7360323886639676.
[I 2025-04-03 15:30:39,949] Trial 3 finished with value: 0.7311740890688259 and parameters: {'w1': 0.9807641983846155, 'w2': 0.6848297385848633}. Best is trial 1 with value: 0.7360323886639676.
[I 2025-04-03 15:30:39,953] Trial 4 finished with value: 0.7327935222672065 and parameters: {'w1': 0.4809319

In [288]:
# 最適な重みの取得（正規化前の値）
best_params9 = study9.best_params

# 最適な trial の user_attrs から正規化後の値を取得
best_trial = study9.best_trial
best_w1 = best_trial.user_attrs["w1_norm"]
best_w2 = best_trial.user_attrs["w2_norm"]
# best_w3 = best_trial.user_attrs["w3_norm"]

# 出力
print("Best Weights (raw):", best_params9)  # 正規化前の値
# print("Best Weights (normalized):", {"w1": best_w1, "w2": best_w2, "w3": best_w3})
print("Best Weights (normalized):", {"w1": best_w1, "w2": best_w2})

Best Weights (raw): {'w1': 0.2742440766772329, 'w2': 0.6338092253155581}
Best Weights (normalized): {'w1': 0.30201319248042346, 'w2': 0.6979868075195764}


In [109]:
stacked_train1 = ensemble_mean_train1.drop(["pred_class", "pred_class_submit"], axis=1)
stacked_test1 = ensemble_mean_test1.drop(["pred_class", "pred_class_submit"], axis=1)

In [110]:
stacked_train1.head(), stacked_train1.shape

(          0         1         2
 0  0.529835  0.315878  0.154288
 1  0.045060  0.672778  0.282162
 2  0.784094  0.067574  0.148332
 3  0.395242  0.293307  0.311451
 4  0.024213  0.009868  0.965919,
 (1235, 3))

In [111]:
# stacked_train2 = pd.concat(
#     [
#         lgb_tr_pred[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]],
#         cat_tr_pred[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]],
#         # xgb_tr_pred[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]],
#     ],
#     axis=1,
# )

# stacked_test2 = pd.concat(
#     [
#         lgb_test_pred[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]],
#         cat_test_pred[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]],
#         # xgb_test_pred[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]],
#     ],
#     axis=1,
# )

stacked_train2 = pd.concat(
    [
        cat_tr_pred1[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]],
        cat_tr_pred2[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]],
    ],
    axis=1,
)
stacked_train2.columns = [
    "クラス_0_cat1",
    "クラス_1_cat1",
    "クラス_2_cat1",
    "クラス_0_cat2",
    "クラス_1_cat2",
    "クラス_2_cat2",
]

stacked_test2 = pd.concat(
    [
        cat_test_pred1[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]],
        cat_test_pred2[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]],
    ],
    axis=1,
)
stacked_test2.columns = [
    "クラス_0_cat1",
    "クラス_1_cat1",
    "クラス_2_cat1",
    "クラス_0_cat2",
    "クラス_1_cat2",
    "クラス_2_cat2",
]

stacked_train2.head(), stacked_train2.shape
stacked_test2.head(), stacked_test2.shape

(   クラス_0_cat1  クラス_1_cat1  クラス_2_cat1  クラス_0_cat2  クラス_1_cat2  クラス_2_cat2
 0    0.010526    0.013270    0.976205    0.012708    0.012696    0.974596
 1    0.763080    0.082711    0.154209    0.727823    0.089642    0.182535
 2    0.108507    0.033956    0.857537    0.115339    0.036195    0.848465
 3    0.027384    0.901426    0.071191    0.030484    0.905408    0.064108
 4    0.097956    0.031178    0.870866    0.115969    0.034675    0.849356,
 (824, 6))

In [112]:
y_train1.head(), y_train1.shape

(0    0.0
 1    1.0
 2    2.0
 3    2.0
 4    2.0
 Name: outcome, dtype: float64,
 (1235,))

#### ロジスティック回帰

In [ ]:
# from sklearn.model_selection import GridSearchCV

# # ロジスティック回帰のハイパーパラメータの候補
# param_grid = {
#     "C": [0.01, 0.1, 1, 10, 100],  # 正則化強度
#     "solver": ["lbfgs"],  # 最適化アルゴリズム
#     "max_iter": [100, 500, 1000],  # 最大反復回数
# }

# # GridSearchCVのインスタンスを作成
# grid_search = GridSearchCV(
#     LogisticRegression(multi_class="multinomial", random_state=123),
#     param_grid,
#     cv=5,
#     n_jobs=-1,
#     scoring="f1_micro",
# )

# # スタッキングモデルの予測確率を使ってメタモデルを学習
# grid_search.fit(stacked_train2, y_train2)

# # 最適なハイパーパラメータ
# best_params = grid_search.best_params_
# print(f"Best hyperparameters: {best_params}")

Best hyperparameters: {'C': 0.1, 'max_iter': 100, 'solver': 'lbfgs'}


In [204]:
# lr_metrics_stack, lr_tr_pred_stack, lr_test_pred_stack = lr_train_eval(
#     stacked_train1, y_train1, stacked_test1, 3, 5
# )

lr_metrics_stack, lr_tr_pred_stack, lr_test_pred_stack = lr_train_eval(
    stacked_train2, y_train1, stacked_test2, 3, 5
)

In [205]:
print("【マイクロF1スコア】")
print(lr_metrics_stack)
print(
    "train：{:.2f} ± {:.2f}".format(
        lr_metrics_stack[:, 1].mean(), lr_metrics_stack[:, 1].std()
    )
)
print(
    "val：{:.2f} ± {:.2f}".format(
        lr_metrics_stack[:, 2].mean(), lr_metrics_stack[:, 2].std()
    )
)

【マイクロF1スコア】
[[0.         0.73785425 0.76923077]
 [1.         0.74089069 0.73279352]
 [2.         0.74493927 0.71659919]
 [3.         0.73481781 0.76518219]
 [4.         0.74089069 0.73279352]]
train：0.74 ± 0.00
val：0.74 ± 0.02


In [206]:
submit = pd.DataFrame(
    data={"id": test_id, "outcome": lr_test_pred_stack["pred_class_submit"]}
)
display(submit.head(), submit.shape)
submit.to_csv("./submit/submit_lr_stack.csv", index=False)

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived


(824, 2)

#### LightGBM

In [113]:
# lgb_metrics_stack, lgb_imp_stack, lgb_tr_pred_stack, lgb_test_pred_stack = (
#     lgb_train_eval(stacked_train1, y_train1, stacked_test1, params_lgb, 5)
# )

lgb_metrics_stack, lgb_imp_stack, lgb_tr_pred_stack, lgb_test_pred_stack = (
    lgb_train_eval(stacked_train2, y_train1, stacked_test2, params_lgb2, 5)
)

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.668128	training's micro_f1: 0.759109	valid_1's multi_logloss: 0.669305	valid_1's micro_f1: 0.744939
Early stopping, best iteration is:
[52]	training's multi_logloss: 0.775663	training's micro_f1: 0.746964	valid_1's multi_logloss: 0.770251	valid_1's micro_f1: 0.753036
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.646397	training's micro_f1: 0.770243	valid_1's multi_logloss: 0.759153	valid_1's micro_f1: 0.704453
[200]	training's multi_logloss: 0.547755	training's micro_f1: 0.776316	valid_1's multi_logloss: 0.718263	valid_1's micro_f1: 0.720648
Early stopping, best iteration is:
[124]	training's multi_logloss: 0.612689	training's micro_f1: 0.771255	valid_1's multi_logloss: 0.741271	valid_1's micro_f1: 0.728745
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.650718	training's micro_f1: 0.768219	vali

In [114]:
print("【マイクロF1スコア】")
print(lgb_metrics_stack)
print(
    "train：{:.2f} ± {:.2f}".format(
        lgb_metrics_stack[:, 1].mean(), lgb_metrics_stack[:, 1].std()
    )
)
print(
    "val：{:.2f} ± {:.2f}".format(
        lgb_metrics_stack[:, 2].mean(), lgb_metrics_stack[:, 2].std()
    )
)

【マイクロF1スコア】
[[0.         0.74696356 0.75303644]
 [1.         0.77125506 0.72874494]
 [2.         0.77024291 0.71659919]
 [3.         0.77125506 0.75708502]
 [4.         0.76315789 0.72874494]]
train：0.76 ± 0.01
val：0.74 ± 0.02


In [116]:
submit = pd.DataFrame(
    data={"id": test_id, "outcome": lgb_test_pred_stack["pred_class_submit"]}
)
display(submit.head(), submit.shape)
submit.to_csv("./submit/submit_lgb_stack.csv", index=False)

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived


(824, 2)

#### XGBoost

In [115]:
# xgb_metrics_stack, xgb_imp_stack, xgb_tr_pred_stack, xgb_test_pred_stack = (
#     xgb_train_eval(stacked_train1, y_train1, stacked_test1, params_xgb, 5)
# )

xgb_metrics_stack, xgb_imp_stack, xgb_tr_pred_stack, xgb_test_pred_stack = (
    xgb_train_eval(stacked_train2, y_train1, stacked_test2, params_xgb2, 5)
)

[0]	validation_0-merror:0.25810	validation_1-merror:0.22672
[1]	validation_0-merror:0.26012	validation_1-merror:0.22267
[2]	validation_0-merror:0.26012	validation_1-merror:0.23077
[3]	validation_0-merror:0.25304	validation_1-merror:0.22672
[4]	validation_0-merror:0.25607	validation_1-merror:0.22267
[5]	validation_0-merror:0.25709	validation_1-merror:0.23077
[6]	validation_0-merror:0.25506	validation_1-merror:0.23482
[7]	validation_0-merror:0.24899	validation_1-merror:0.23482
[8]	validation_0-merror:0.24696	validation_1-merror:0.23482
[9]	validation_0-merror:0.24798	validation_1-merror:0.23887
[10]	validation_0-merror:0.24798	validation_1-merror:0.23077
[11]	validation_0-merror:0.24798	validation_1-merror:0.23077
[12]	validation_0-merror:0.25000	validation_1-merror:0.23077
[13]	validation_0-merror:0.24899	validation_1-merror:0.23077
[14]	validation_0-merror:0.24798	validation_1-merror:0.23077
[15]	validation_0-merror:0.25101	validation_1-merror:0.23482
[16]	validation_0-merror:0.25000	v

In [119]:
print("【マイクロF1スコア】")
print(xgb_metrics_stack)
print(
    "train：{:.2f} ± {:.2f}".format(
        xgb_metrics_stack[:, 1].mean(), xgb_metrics_stack[:, 1].std()
    )
)
print(
    "val：{:.2f} ± {:.2f}".format(
        xgb_metrics_stack[:, 2].mean(), xgb_metrics_stack[:, 2].std()
    )
)

【マイクロF1スコア】
[[0.         0.73987854 0.77732794]
 [1.         0.77226721 0.74493927]
 [2.         0.75910931 0.72064777]
 [3.         0.75202429 0.76923077]
 [4.         0.75910931 0.74493927]]
train：0.76 ± 0.01
val：0.75 ± 0.02


In [121]:
submit = pd.DataFrame(
    data={"id": test_id, "outcome": xgb_test_pred_stack["pred_class_submit"]}
)
display(submit.head(), submit.shape)
submit.to_csv("./submit/submit_xgb_stack.csv", index=False)

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived


(824, 2)

#### CatBoost

In [195]:
category_cols_cat = []
# cat_metrics_stack, cat_imp_stack, cat_tr_pred_stack, cat_test_pred_stack = (
#     cat_train_eval(
#         stacked_train1, y_train1, stacked_test1, category_cols_cat, params_cat2, 5
#     )
# )

cat_metrics_stack, cat_imp_stack, cat_tr_pred_stack, cat_test_pred_stack = (
    cat_train_eval(
        stacked_train2, y_train1, stacked_test2, category_cols_cat, params_cat4, 5
    )
)

0:	learn: 1.0224268	test: 1.0136645	best: 1.0136645 (0)	total: 13.2ms	remaining: 2m 11s
100:	learn: 0.5445597	test: 0.6032204	best: 0.5993691 (93)	total: 143ms	remaining: 14s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.599369068
bestIteration = 93

Shrink model to first 94 iterations.
0:	learn: 1.0196785	test: 1.0282279	best: 1.0282279 (0)	total: 3.05ms	remaining: 30.5s
100:	learn: 0.5244645	test: 0.6993878	best: 0.6956691 (82)	total: 94.9ms	remaining: 9.3s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.69566908
bestIteration = 82

Shrink model to first 83 iterations.
0:	learn: 1.0197767	test: 1.0261563	best: 1.0261563 (0)	total: 1.01ms	remaining: 10.1s
100:	learn: 0.5278487	test: 0.6913311	best: 0.6898491 (94)	total: 112ms	remaining: 11s
200:	learn: 0.4310211	test: 0.6968626	best: 0.6881332 (129)	total: 211ms	remaining: 10.3s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6881332428
bestIteration = 129

Shrink model 

In [196]:
print("【マイクロF1スコア】")
print(cat_metrics_stack)
print(
    "train：{:.2f} ± {:.2f}".format(
        cat_metrics_stack[:, 1].mean(), cat_metrics_stack[:, 1].std()
    )
)
print(
    "val：{:.2f} ± {:.2f}".format(
        cat_metrics_stack[:, 2].mean(), cat_metrics_stack[:, 2].std()
    )
)

【マイクロF1スコア】
[[0.         0.76518219 0.75708502]
 [1.         0.77125506 0.74089069]
 [2.         0.78947368 0.72469636]
 [3.         0.75303644 0.75708502]
 [4.         0.75       0.73684211]]
train：0.77 ± 0.01
val：0.74 ± 0.01


In [181]:
submit = pd.DataFrame(
    data={"id": test_id, "outcome": cat_test_pred_stack["pred_class_submit"]}
)
display(submit.head(), submit.shape)
submit.to_csv("./submit/submit_cat_stack.csv", index=False)

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived


(824, 2)

#### ランダムフォレスト

In [ ]:
rf_metrics_stack, rf_imp_stack, rf_tr_pred_stack, rf_test_pred_stack = rf_train_eval(
    stacked_train1, y_train1, stacked_test1, params_rf, 5
)

# rf_metrics_stack, rf_imp_stack, rf_tr_pred_stack, rf_test_pred_stack = rf_train_eval(
#     stacked_train2, y_train1, stacked_test2, params_rf, 5
# )

In [135]:
print("【マイクロF1スコア】")
print(rf_metrics_stack)
print(
    "train：{:.2f} ± {:.2f}".format(
        rf_metrics_stack[:, 1].mean(), rf_metrics_stack[:, 1].std()
    )
)
print(
    "val：{:.2f} ± {:.2f}".format(
        rf_metrics_stack[:, 2].mean(), rf_metrics_stack[:, 2].std()
    )
)

【マイクロF1スコア】
[[0.         0.85020243 0.75303644]
 [1.         0.85425101 0.68825911]
 [2.         0.84311741 0.67611336]
 [3.         0.85526316 0.72469636]
 [4.         0.83097166 0.70445344]]
train：0.85 ± 0.01
val：0.71 ± 0.03


#### アンサンブル(2段階目)

In [ ]:
### アンサンブル

w1 = 0.9960683931819493
w2 = 0.003931606818050753

ensemble_mean_stack_train1 = pd.DataFrame(
    w1 * xgb_tr_pred_stack[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
    + w2 * lgb_tr_pred_stack[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
)
ensemble_mean_stack_train1["pred_class"] = ensemble_mean_stack_train1.idxmax(axis=1)

ensemble_mean_stack_test1 = pd.DataFrame(
    w1 * xgb_test_pred_stack[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
    + w2 * lgb_test_pred_stack[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
)
ensemble_mean_stack_test1["pred_class"] = ensemble_mean_stack_test1.idxmax(axis=1)

# ensemble_mean_stack_train1 = (
#     pd.DataFrame(
#         lgb_tr_pred_stack[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
#         + xgb_tr_pred_stack[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
#     )
#     / 2
# )
# ensemble_mean_stack_train1["pred_class"] = ensemble_mean_stack_train1.idxmax(axis=1)

# ensemble_mean_stack_test1 = (
#     pd.DataFrame(
#         lgb_test_pred_stack[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
#         + xgb_test_pred_stack[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
#     )
#     / 2
# )
# ensemble_mean_stack_test1["pred_class"] = ensemble_mean_stack_test1.idxmax(axis=1)


# w1 = 0.9691823560464305
# w2 = 0.0013747498832208612
# w3 = 0.029442894070348634

# w1 = 0.9938063382602554
# w3 = 0.006193661739744571

# ensemble_mean_stack_train1 = pd.DataFrame(
#     w1 * xgb_tr_pred_stack[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
#     # + w2 * lgb_tr_pred_stack[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
#     + w3 * cat_tr_pred_stack[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
# )
# ensemble_mean_stack_train1["pred_class"] = ensemble_mean_stack_train1.idxmax(axis=1)

# ensemble_mean_stack_test1 = pd.DataFrame(
#     w1 * xgb_test_pred_stack[["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]].values
#     # + w2 * lgb_test_pred_stack[["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]].values
#     + w3 * cat_test_pred_stack[["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]].values
# )
# ensemble_mean_stack_test1["pred_class"] = ensemble_mean_stack_test1.idxmax(axis=1)

# ラベルを元に戻すための辞書
label_map = {0: "died", 1: "euthanized", 2: "lived"}

ensemble_mean_stack_train1["pred_class_submit"] = ensemble_mean_stack_train1[
    "pred_class"
].map(label_map)
ensemble_mean_stack_test1["pred_class_submit"] = ensemble_mean_stack_test1[
    "pred_class"
].map(label_map)

# 精度を確認
print(
    "マイクロF1スコア:{:.5f}".format(
        f1_score(y_train1, ensemble_mean_stack_train1["pred_class"], average="micro")
    )
)

マイクロF1スコア:0.75142


In [211]:
submit = pd.DataFrame(
    data={"id": test_id, "outcome": ensemble_mean_stack_test1["pred_class_submit"]}
)
display(submit.head(), submit.shape, submit.isnull().sum())

submit.to_csv("./submit/submit_ensemble_stack1.csv", index=False)

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived


(824, 2)

id         0
outcome    0
dtype: int64

In [203]:
model_predict1 = xgb_tr_pred_stack[
    ["クラス_0_xgb", "クラス_1_xgb", "クラス_2_xgb"]
].values
# model_predict2 = lgb_tr_pred_stack[
#     ["クラス_0_lgb", "クラス_1_lgb", "クラス_2_lgb"]
# ].values

model_predict2 = cat_tr_pred_stack[
    ["クラス_0_cat", "クラス_1_cat", "クラス_2_cat"]
].values
y_true = y_train2

sampler = optuna.samplers.TPESampler(seed=123)
study10 = optuna.create_study(sampler=sampler, direction="maximize")
study10.optimize(objective_ensemble_weight, n_trials=100)

[I 2025-04-03 14:10:37,356] A new study created in memory with name: no-name-05958736-090a-4426-be2d-90d676371a52
[I 2025-04-03 14:10:37,363] Trial 0 finished with value: 0.7441295546558705 and parameters: {'w1': 0.6964691855978616, 'w2': 0.28613933495037946}. Best is trial 0 with value: 0.7441295546558705.
[I 2025-04-03 14:10:37,370] Trial 1 finished with value: 0.7408906882591093 and parameters: {'w1': 0.2268514535642031, 'w2': 0.5513147690828912}. Best is trial 0 with value: 0.7441295546558705.
[I 2025-04-03 14:10:37,374] Trial 2 finished with value: 0.7433198380566801 and parameters: {'w1': 0.7194689697855631, 'w2': 0.42310646012446096}. Best is trial 0 with value: 0.7441295546558705.
[I 2025-04-03 14:10:37,379] Trial 3 finished with value: 0.7425101214574898 and parameters: {'w1': 0.9807641983846155, 'w2': 0.6848297385848633}. Best is trial 0 with value: 0.7441295546558705.
[I 2025-04-03 14:10:37,383] Trial 4 finished with value: 0.7433198380566801 and parameters: {'w1': 0.4809319

In [206]:
# 最適な重みの取得（正規化前の値）
best_params10 = study10.best_params

# 最適な trial の user_attrs から正規化後の値を取得
best_trial = study10.best_trial
best_w1 = best_trial.user_attrs["w1_norm"]
best_w2 = best_trial.user_attrs["w2_norm"]
# best_w3 = best_trial.user_attrs["w3_norm"]

# 出力
print("Best Weights (raw):", best_params10)  # 正規化前の値
# print("Best Weights (normalized):", {"w1": best_w1, "w2": best_w2, "w3": best_w3})
print("Best Weights (normalized):", {"w1": best_w1, "w2": best_w2})

Best Weights (raw): {'w1': 0.6794996805319204, 'w2': 0.004234820217434636}
Best Weights (normalized): {'w1': 0.9938063382602554, 'w2': 0.006193661739744571}


#### Oputuna(LightGBM)

In [173]:
x_train = stacked_train2
y_train = y_train2
x_test = stacked_test2

sampler = optuna.samplers.TPESampler(seed=123)
study6 = optuna.create_study(sampler=sampler, direction="maximize")
study6.optimize(objective_lgb, n_trials=30)

[I 2025-04-02 18:47:29,037] A new study created in memory with name: no-name-a5345cad-05d7-47c5-8fbf-6f9450a1f2b6
/tmp/ipykernel_429/1705590420.py:39: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "min_gain_to_split": trial.suggest_loguniform("min_gain_to_split", 1e-8, 1.0),


Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.3869	training's micro_f1: 0.860324	valid_1's multi_logloss: 0.658155	valid_1's micro_f1: 0.748988
Early stopping, best iteration is:
[32]	training's multi_logloss: 0.546053	training's micro_f1: 0.771255	valid_1's multi_logloss: 0.59935	valid_1's micro_f1: 0.765182
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.363501	training's micro_f1: 0.860324	valid_1's multi_logloss: 0.784484	valid_1's micro_f1: 0.676113
Early stopping, best iteration is:
[9]	training's multi_logloss: 0.677556	training's micro_f1: 0.752024	valid_1's multi_logloss: 0.760376	valid_1's micro_f1: 0.732794
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.367967	training's micro_f1: 0.867409	valid_1's multi_logloss: 0.775766	valid_1's micro_f1: 0.696356
Early stopping, best iteration is:
[19]	training's multi_logloss: 0.5797	trainin

[I 2025-04-02 18:47:54,130] Trial 0 finished with value: 0.740080971659919 and parameters: {'min_child_samples': 37, 'max_depth': 5, 'bagging_fraction': 0.65, 'feature_fraction': 0.8, 'min_gain_to_split': 0.005698384608345678}. Best is trial 0 with value: 0.740080971659919.


Early stopping, best iteration is:
[8]	training's multi_logloss: 0.697193	training's micro_f1: 0.757085	valid_1's multi_logloss: 0.75903	valid_1's micro_f1: 0.724696
Training until validation scores don't improve for 100 rounds


/tmp/ipykernel_429/1705590420.py:39: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "min_gain_to_split": trial.suggest_loguniform("min_gain_to_split", 1e-8, 1.0),


...

Early stopping, best iteration is:
[9]	training's multi_logloss: 0.682524	training's micro_f1: 0.753036	valid_1's multi_logloss: 0.739388	valid_1's micro_f1: 0.736842
Training until validation scores don't improve for 100 rounds


/tmp/ipykernel_429/1705590420.py:39: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "min_gain_to_split": trial.suggest_loguniform("min_gain_to_split", 1e-8, 1.0),


[100]	training's multi_logloss: 0.359748	training's micro_f1: 0.879555	valid_1's multi_logloss: 0.665293	valid_1's micro_f1: 0.757085
Early stopping, best iteration is:
[30]	training's multi_logloss: 0.544542	training's micro_f1: 0.774291	valid_1's multi_logloss: 0.603348	valid_1's micro_f1: 0.769231
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.340916	training's micro_f1: 0.87753	valid_1's multi_logloss: 0.780011	valid_1's micro_f1: 0.672065
Early stopping, best iteration is:
[11]	training's multi_logloss: 0.647718	training's micro_f1: 0.757085	valid_1's multi_logloss: 0.73918	valid_1's micro_f1: 0.736842
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.345006	training's micro_f1: 0.879555	valid_1's multi_logloss: 0.797739	valid_1's micro_f1: 0.692308
Early stopping, best iteration is:
[18]	training's multi_logloss: 0.584692	training's micro_f1: 0.762146	valid_1's multi_logloss: 0.702872	v

[I 2025-04-02 18:57:57,820] Trial 29 finished with value: 0.7425101214574898 and parameters: {'min_child_samples': 37, 'max_depth': 7, 'bagging_fraction': 0.65, 'feature_fraction': 0.6, 'min_gain_to_split': 0.017330883508858894}. Best is trial 12 with value: 0.7441295546558704.


Early stopping, best iteration is:
[7]	training's multi_logloss: 0.719932	training's micro_f1: 0.75	valid_1's multi_logloss: 0.771808	valid_1's micro_f1: 0.724696


In [174]:
# 結果
trial6 = study6.best_trial
print("マイクロF1スコア(best)：{:.4f}".format(trial6.value))
print("ベストパラメータ：{}".format(trial6.params))

マイクロF1スコア(best)：0.7441
ベストパラメータ：{'min_child_samples': 47, 'max_depth': 3, 'bagging_fraction': 0.7, 'feature_fraction': 0.9, 'min_gain_to_split': 1.1797476912384551e-08}


In [234]:
params_stack_best_lgb = trial6.params
params_stack_best_lgb.update(params_base_lgb)  # ベースパラメータを追加
display(params_stack_best_lgb)

{'learning_rate': 0.1,
 'min_child_samples': 18,
 'max_depth': 4,
 'bagging_fraction': 0.8,
 'feature_fraction': 0.85,
 'min_gain_to_split': 2.4257815076675944e-05,
 'boosting_type': 'gbdt',
 'objective': 'multiclass',
 'num_class': 3,
 'metric': 'multi_logloss',
 'importance_type': 'gain',
 'random_state': 42,
 'bagging_freq': 1,
 'n_estimators': 20000,
 'lambda_l1': 0.0,
 'lambda_l2': 1.0,
 'verbose': -1}

In [239]:
(
    lgb_metrics_stack_oputuna,
    lgb_imp_stack_oputuna,
    lgb_tr_pred_stack_oputuna,
    lgb_test_pred_stack_oputuna,
) = lgb_train_eval(stacked_train2, y_train2, stacked_test2, params_stack_best_lgb, 5)

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.359425	training's micro_f1: 0.865385	valid_1's multi_logloss: 0.678889	valid_1's micro_f1: 0.732794
Early stopping, best iteration is:
[25]	training's multi_logloss: 0.547937	training's micro_f1: 0.777328	valid_1's multi_logloss: 0.614971	valid_1's micro_f1: 0.769231
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.331943	training's micro_f1: 0.876518	valid_1's multi_logloss: 0.76351	valid_1's micro_f1: 0.659919
Early stopping, best iteration is:
[19]	training's multi_logloss: 0.556214	training's micro_f1: 0.770243	valid_1's multi_logloss: 0.715208	valid_1's micro_f1: 0.736842
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.338544	training's micro_f1: 0.879555	valid_1's multi_logloss: 0.769442	valid_1's micro_f1: 0.720648
Early stopping, best iteration is:
[33]	training's multi_logloss: 0.497943	tr

In [240]:
print("【マイクロF1スコア】")
print(lgb_metrics_stack_oputuna)
print(
    "train：{:.2f} ± {:.2f}".format(
        lgb_metrics_stack_oputuna[:, 1].mean(), lgb_metrics_stack_oputuna[:, 1].std()
    )
)
print(
    "val：{:.2f} ± {:.2f}".format(
        lgb_metrics_stack_oputuna[:, 2].mean(), lgb_metrics_stack_oputuna[:, 2].std()
    )
)

【マイクロF1スコア】
[[0.         0.77732794 0.76923077]
 [1.         0.77024291 0.73684211]
 [2.         0.78340081 0.72469636]
 [3.         0.77327935 0.76518219]
 [4.         0.77226721 0.74493927]]
train：0.78 ± 0.00
val：0.75 ± 0.02


#### Oputuna(XGBoost)

In [148]:
x_train = stacked_train2
y_train = y_train2
x_test = stacked_test2

sampler = optuna.samplers.TPESampler(seed=123)
study4 = optuna.create_study(sampler=sampler, direction="maximize")
# study4 = optuna.create_study(sampler=sampler, direction="miniimize")
study4.optimize(objective_xgb, n_trials=30)

[I 2025-04-02 18:28:37,740] A new study created in memory with name: no-name-5ec91f01-07ba-4540-9b69-b9862850e55a


[0]	validation_0-merror:0.24595	validation_1-merror:0.23887
[1]	validation_0-merror:0.24089	validation_1-merror:0.23887
[2]	validation_0-merror:0.23178	validation_1-merror:0.25506
[3]	validation_0-merror:0.23178	validation_1-merror:0.23887
[4]	validation_0-merror:0.23178	validation_1-merror:0.24696
[5]	validation_0-merror:0.23178	validation_1-merror:0.25506


/tmp/ipykernel_429/2421624824.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "eta": trial.suggest_loguniform("eta", 0.01, 0.3),
/tmp/ipykernel_429/2421624824.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.1, 10),
/tmp/ipykernel_429/2421624824.py:35: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "gamma": trial.suggest_loguniform("gamma", 1e-8, 1.0),


[6]	validation_0-merror:0.22470	validation_1-merror:0.26316
[7]	validation_0-merror:0.21457	validation_1-merror:0.25101
[8]	validation_0-merror:0.21660	validation_1-merror:0.25506
[9]	validation_0-merror:0.22065	validation_1-merror:0.25911
[10]	validation_0-merror:0.22166	validation_1-merror:0.25911
[11]	validation_0-merror:0.22267	validation_1-merror:0.25101
[12]	validation_0-merror:0.22470	validation_1-merror:0.25506
[13]	validation_0-merror:0.22166	validation_1-merror:0.25911
[14]	validation_0-merror:0.22065	validation_1-merror:0.26316
[15]	validation_0-merror:0.22166	validation_1-merror:0.25911
[16]	validation_0-merror:0.22166	validation_1-merror:0.25101
[17]	validation_0-merror:0.22166	validation_1-merror:0.25101
[18]	validation_0-merror:0.22065	validation_1-merror:0.25101
[19]	validation_0-merror:0.21862	validation_1-merror:0.24291
[20]	validation_0-merror:0.21761	validation_1-merror:0.24291
[21]	validation_0-merror:0.21660	validation_1-merror:0.24291
[22]	validation_0-merror:0.2

[I 2025-04-02 18:28:52,502] Trial 0 finished with value: 0.7433198380566802 and parameters: {'eta': 0.10684874790620232, 'min_child_weight': 0.37348973478010355, 'max_depth': 4, 'subsample': 0.8, 'colsample_bytree': 0.85, 'gamma': 2.4257815076675944e-05}. Best is trial 0 with value: 0.7433198380566802.


...

[I 2025-04-02 18:39:43,739] Trial 28 finished with value: 0.7408906882591093 and parameters: {'eta': 0.07248185048673499, 'min_child_weight': 4.351901518494499, 'max_depth': 7, 'subsample': 0.6, 'colsample_bytree': 0.65, 'gamma': 2.5713833665797e-08}. Best is trial 10 with value: 0.7465587044534413.


[0]	validation_0-merror:0.14879	validation_1-merror:0.29150
[1]	validation_0-merror:0.12247	validation_1-merror:0.27126


/tmp/ipykernel_429/2421624824.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "eta": trial.suggest_loguniform("eta", 0.01, 0.3),
/tmp/ipykernel_429/2421624824.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.1, 10),
/tmp/ipykernel_429/2421624824.py:35: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "gamma": trial.suggest_loguniform("gamma", 1e-8, 1.0),


[2]	validation_0-merror:0.11336	validation_1-merror:0.25101
[3]	validation_0-merror:0.10526	validation_1-merror:0.25911
[4]	validation_0-merror:0.09919	validation_1-merror:0.26316
[5]	validation_0-merror:0.09615	validation_1-merror:0.25911
[6]	validation_0-merror:0.09109	validation_1-merror:0.25911
[7]	validation_0-merror:0.08401	validation_1-merror:0.26316
[8]	validation_0-merror:0.08502	validation_1-merror:0.25101
[9]	validation_0-merror:0.07287	validation_1-merror:0.25101
[10]	validation_0-merror:0.06781	validation_1-merror:0.24291
[11]	validation_0-merror:0.06377	validation_1-merror:0.23887
[12]	validation_0-merror:0.05972	validation_1-merror:0.24696
[13]	validation_0-merror:0.05567	validation_1-merror:0.25101
[14]	validation_0-merror:0.05263	validation_1-merror:0.25506
[15]	validation_0-merror:0.04656	validation_1-merror:0.25101
[16]	validation_0-merror:0.04352	validation_1-merror:0.25506
[17]	validation_0-merror:0.03543	validation_1-merror:0.25101
[18]	validation_0-merror:0.03745

[I 2025-04-02 18:40:08,879] Trial 29 finished with value: 0.7238866396761134 and parameters: {'eta': 0.12245771219927316, 'min_child_weight': 0.20354816423272123, 'max_depth': 9, 'subsample': 0.8, 'colsample_bytree': 0.6, 'gamma': 2.0187394941345541e-07}. Best is trial 10 with value: 0.7465587044534413.


In [149]:
# 結果
trial4 = study4.best_trial
print("マイクロF1スコア(best)：{:.4f}".format(trial4.value))
print("ベストパラメータ：{}".format(trial4.params))

マイクロF1スコア(best)：0.7466
ベストパラメータ：{'eta': 0.013951361469852156, 'min_child_weight': 9.870814292978041, 'max_depth': 9, 'subsample': 0.6, 'colsample_bytree': 0.6, 'gamma': 1.1262297460223246e-08}


In [150]:
params_stack_best_xgb = trial4.params
params_stack_best_xgb.update(params_base_xgb)  # ベースパラメータを追加
display(params_stack_best_xgb)

{'eta': 0.013951361469852156,
 'min_child_weight': 9.870814292978041,
 'max_depth': 9,
 'subsample': 0.6,
 'colsample_bytree': 0.6,
 'gamma': 1.1262297460223246e-08,
 'objective': 'multi:softprob',
 'num_class': 3,
 'booster': 'gbtree',
 'eval_metric': 'merror',
 'random_state': 123,
 'importance_type': 'total_gain',
 'early_stopping_rounds': 100,
 'n_estimators': 10000,
 'alpha': 0.0,
 'lambda': 1.0}

In [151]:
(
    xgb_metrics_stack_oputuna,
    xgb_imp_stack_oputuna,
    xgb_tr_pred_stack_oputuna,
    xgb_test_pred_stack_oputuna,
) = xgb_train_eval(stacked_train2, y_train2, stacked_test2, params_stack_best_xgb, 5)

[0]	validation_0-merror:0.25810	validation_1-merror:0.22672


[1]	validation_0-merror:0.26012	validation_1-merror:0.22267
[2]	validation_0-merror:0.26012	validation_1-merror:0.23077
[3]	validation_0-merror:0.25304	validation_1-merror:0.22672
[4]	validation_0-merror:0.25607	validation_1-merror:0.22267
[5]	validation_0-merror:0.25709	validation_1-merror:0.23077
[6]	validation_0-merror:0.25506	validation_1-merror:0.23482
[7]	validation_0-merror:0.24899	validation_1-merror:0.23482
[8]	validation_0-merror:0.24696	validation_1-merror:0.23482
[9]	validation_0-merror:0.24798	validation_1-merror:0.23887
[10]	validation_0-merror:0.24798	validation_1-merror:0.23077
[11]	validation_0-merror:0.24798	validation_1-merror:0.23077
[12]	validation_0-merror:0.25000	validation_1-merror:0.23077
[13]	validation_0-merror:0.24899	validation_1-merror:0.23077
[14]	validation_0-merror:0.24798	validation_1-merror:0.23077
[15]	validation_0-merror:0.25101	validation_1-merror:0.23482
[16]	validation_0-merror:0.25000	validation_1-merror:0.23077
[17]	validation_0-merror:0.25202	

In [152]:
print("【マイクロF1スコア】")
print(xgb_metrics_stack_oputuna)
print(
    "train：{:.2f} ± {:.2f}".format(
        xgb_metrics_stack_oputuna[:, 1].mean(), xgb_metrics_stack_oputuna[:, 1].std()
    )
)
print(
    "val：{:.2f} ± {:.2f}".format(
        xgb_metrics_stack_oputuna[:, 2].mean(), xgb_metrics_stack_oputuna[:, 2].std()
    )
)

【マイクロF1スコア】
[[0.         0.73987854 0.77732794]
 [1.         0.77226721 0.74493927]
 [2.         0.75910931 0.72064777]
 [3.         0.75202429 0.76923077]
 [4.         0.75910931 0.74493927]]
train：0.76 ± 0.01
val：0.75 ± 0.02


#### Oputuna(CatBoost)

In [154]:
x_train = stacked_train2
y_train = y_train2
x_test = stacked_test2
categorical_features = []

sampler = optuna.samplers.TPESampler(seed=123)
study5 = optuna.create_study(sampler=sampler, direction="maximize")
study5.optimize(objective_cat, n_trials=30)

[I 2025-04-03 13:51:02,002] A new study created in memory with name: no-name-0469addb-f6d6-4292-b3d4-fa859b0127c6


0:	learn: 1.0224268	test: 1.0136645	best: 1.0136645 (0)	total: 2.32ms	remaining: 23.2s
100:	learn: 0.5445597	test: 0.6032204	best: 0.5993691 (93)	total: 116ms	remaining: 11.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.599369068
bestIteration = 93

Shrink model to first 94 iterations.
0:	learn: 1.0196785	test: 1.0282279	best: 1.0282279 (0)	total: 866us	remaining: 8.67s
100:	learn: 0.5244645	test: 0.6993878	best: 0.6956691 (82)	total: 82.7ms	remaining: 8.1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.69566908
bestIteration = 82

Shrink model to first 83 iterations.
0:	learn: 1.0197767	test: 1.0261563	best: 1.0261563 (0)	total: 5.69ms	remaining: 56.9s
100:	learn: 0.5278487	test: 0.6913311	best: 0.6898491 (94)	total: 105ms	remaining: 10.3s
200:	learn: 0.4310211	test: 0.6968626	best: 0.6881332 (129)	total: 195ms	remaining: 9.51s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6881332428
bestIteration = 129

Shrink mode

[I 2025-04-03 13:51:03,062] Trial 0 finished with value: 0.7433198380566802 and parameters: {'learning_rate': 0.10684874790620232, 'depth': 5, 'bagging_temperature': 0.2268514535642031, 'border_count': 155, 'min_data_in_leaf': 36, 'colsample_bylevel': 0.7115532300622305}. Best is trial 0 with value: 0.7433198380566802.


Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6346198699
bestIteration = 59

Shrink model to first 60 iterations.
0:	learn: 1.0197215	test: 1.0204864	best: 1.0204864 (0)	total: 1.73ms	remaining: 17.3s
100:	learn: 0.5239187	test: 0.6857877	best: 0.6778475 (40)	total: 94.1ms	remaining: 9.22s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6778474672
bestIteration = 40

Shrink model to first 41 iterations.
0:	learn: 0.9256230	test: 0.8982323	best: 0.8982323 (0)	total: 3.53ms	remaining: 35.3s
100:	learn: 0.2803611	test: 0.6555457	best: 0.6041220 (20)	total: 326ms	remaining: 31.9s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6041220342
bestIteration = 20

Shrink model to first 21 iterations.
0:	learn: 0.9082336	test: 0.9434026	best: 0.9434026 (0)	total: 2.21ms	remaining: 22.1s
100:	learn: 0.2827701	test: 0.7844671	best: 0.7126120 (13)	total: 404ms	remaining: 39.6s
Stopped by overfitting detector  (100 iterations wait)

best

[I 2025-04-03 13:51:05,144] Trial 1 finished with value: 0.7271255060728745 and parameters: {'learning_rate': 0.281000856274069, 'depth': 8, 'bagging_temperature': 0.48093190148436094, 'border_count': 119, 'min_data_in_leaf': 18, 'colsample_bylevel': 0.8645248536920208}. Best is trial 0 with value: 0.7433198380566802.


...

100:	learn: 0.5560123	test: 0.6937854	best: 0.6855933 (56)	total: 144ms	remaining: 14.1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.685593277
bestIteration = 56

Shrink model to first 57 iterations.
0:	learn: 1.0770516	test: 1.0735354	best: 1.0735354 (0)	total: 883us	remaining: 8.83s
100:	learn: 0.6378944	test: 0.6116128	best: 0.6116128 (100)	total: 103ms	remaining: 10.1s
200:	learn: 0.5926741	test: 0.5983329	best: 0.5980442 (191)	total: 191ms	remaining: 9.32s
300:	learn: 0.5585949	test: 0.6004147	best: 0.5979496 (229)	total: 276ms	remaining: 8.91s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.5979495868
bestIteration = 229

Shrink model to first 230 iterations.
0:	learn: 1.0756506	test: 1.0789582	best: 1.0789582 (0)	total: 809us	remaining: 8.1s
100:	learn: 0.6147772	test: 0.7051118	best: 0.7051118 (100)	total: 87.6ms	remaining: 8.58s
200:	learn: 0.5710180	test: 0.6998982	best: 0.6990199 (171)	total: 176ms	remaining: 8.59s
300:	learn: 0.

[I 2025-04-03 13:52:17,727] Trial 28 finished with value: 0.7392712550607288 and parameters: {'learning_rate': 0.03057551935571084, 'depth': 5, 'bagging_temperature': 0.00013563804042582284, 'border_count': 155, 'min_data_in_leaf': 47, 'colsample_bylevel': 0.9172154890648561}. Best is trial 0 with value: 0.7433198380566802.


100:	learn: 0.6193489	test: 0.6792196	best: 0.6792196 (100)	total: 97.3ms	remaining: 9.53s
200:	learn: 0.5709626	test: 0.6820439	best: 0.6764691 (116)	total: 186ms	remaining: 9.07s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6764690625
bestIteration = 116

Shrink model to first 117 iterations.
0:	learn: 0.9552390	test: 0.9354720	best: 0.9354720 (0)	total: 6.01ms	remaining: 1m
100:	learn: 0.3034799	test: 0.6233490	best: 0.5936311 (26)	total: 678ms	remaining: 1m 6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.5936310806
bestIteration = 26

Shrink model to first 27 iterations.
0:	learn: 0.9434569	test: 0.9720856	best: 0.9720856 (0)	total: 5.51ms	remaining: 55.1s
100:	learn: 0.2940098	test: 0.7483734	best: 0.6976929 (20)	total: 590ms	remaining: 57.8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6976928767
bestIteration = 20

Shrink model to first 21 iterations.
0:	learn: 0.9447248	test: 0.9544372	best: 0.9544372 (0)	t

[I 2025-04-03 13:52:21,568] Trial 29 finished with value: 0.7303643724696356 and parameters: {'learning_rate': 0.22949634874849845, 'depth': 9, 'bagging_temperature': 0.5559487007789505, 'border_count': 117, 'min_data_in_leaf': 27, 'colsample_bylevel': 0.8667807013887359}. Best is trial 0 with value: 0.7433198380566802.


100:	learn: 0.2952511	test: 0.7441893	best: 0.6963637 (12)	total: 586ms	remaining: 57.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6963636617
bestIteration = 12

Shrink model to first 13 iterations.


In [155]:
# 結果
trial5 = study5.best_trial
print("マイクロF1スコア(best)：{:.4f}".format(trial5.value))
print("ベストパラメータ：{}".format(trial5.params))

マイクロF1スコア(best)：0.7433
ベストパラメータ：{'learning_rate': 0.10684874790620232, 'depth': 5, 'bagging_temperature': 0.2268514535642031, 'border_count': 155, 'min_data_in_leaf': 36, 'colsample_bylevel': 0.7115532300622305}


In [156]:
params_best_cat = trial5.params
params_best_cat.update(params_base_cat)  # ベースパラメータを追加
display(params_best_cat)

{'learning_rate': 0.10684874790620232,
 'depth': 5,
 'bagging_temperature': 0.2268514535642031,
 'border_count': 155,
 'min_data_in_leaf': 36,
 'colsample_bylevel': 0.7115532300622305,
 'loss_function': 'MultiClass',
 'eval_metric': 'MultiClass',
 'iterations': 10000,
 'early_stopping_rounds': 100,
 'random_state': 123,
 'verbose': 0}

In [157]:
(
    cat_metrics_stack_oputuna,
    cat_imp_stack_oputuna,
    cat_tr_pred_stack_oputuna,
    cat_test_pred_stack_oputuna,
) = cat_train_eval(
    stacked_train2, y_train2, stacked_test2, categorical_features, params_best_cat, 5
)

0:	learn: 1.0224268	test: 1.0136645	best: 1.0136645 (0)	total: 4.89ms	remaining: 48.9s
100:	learn: 0.5445597	test: 0.6032204	best: 0.5993691 (93)	total: 114ms	remaining: 11.2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.599369068
bestIteration = 93

Shrink model to first 94 iterations.
0:	learn: 1.0196785	test: 1.0282279	best: 1.0282279 (0)	total: 827us	remaining: 8.27s
100:	learn: 0.5244645	test: 0.6993878	best: 0.6956691 (82)	total: 107ms	remaining: 10.5s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.69566908
bestIteration = 82

Shrink model to first 83 iterations.
0:	learn: 1.0197767	test: 1.0261563	best: 1.0261563 (0)	total: 1.04ms	remaining: 10.4s
100:	learn: 0.5278487	test: 0.6913311	best: 0.6898491 (94)	total: 96.4ms	remaining: 9.44s
200:	learn: 0.4310211	test: 0.6968626	best: 0.6881332 (129)	total: 181ms	remaining: 8.82s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6881332428
bestIteration = 129

Shrink mod

In [158]:
print("===== CatBoost =====")
print("【マイクロF1スコア】")
print(cat_metrics_stack_oputuna)
print(
    "train：{:.2f} ± {:.2f}".format(
        cat_metrics_stack_oputuna[:, 1].mean(), cat_metrics_stack_oputuna[:, 1].std()
    )
)
print(
    "val：{:.2f} ± {:.2f}".format(
        cat_metrics_stack_oputuna[:, 2].mean(), cat_metrics_stack_oputuna[:, 2].std()
    )
)

===== CatBoost =====
【マイクロF1スコア】
[[0.         0.76518219 0.75708502]
 [1.         0.77125506 0.74089069]
 [2.         0.78947368 0.72469636]
 [3.         0.75303644 0.75708502]
 [4.         0.75       0.73684211]]
train：0.77 ± 0.01
val：0.74 ± 0.01


In [159]:
submit = pd.DataFrame(
    data={"id": test_id, "outcome": cat_test_pred_stack["pred_class_submit"]}
)
display(submit.head(), submit.shape)
submit.to_csv("./submit/submit_cat_stack.csv", index=False)

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived


(824, 2)

## 結果の可視化